## Missense Mutation Analysis in ACE2 RBD
Author: Lilli Schuckert \
Date: Last updated 22.05.2024 14:56 CET \
Availability: This code is available on GitHub https://github.com/slilli/Protein-Sequence-Space under the MIT license.

This script analyses the genetically possible variants when taking genetic code redundancy into account. \

Install and import the following dependencies:

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
import random
from Bio import SeqIO
from collections import Counter
import csv
import argparse
import math
from pathlib import Path

from scipy.ndimage import gaussian_filter1d

from Bio.PDB import PDBParser
from Bio.PDB.Superimposer import Superimposer
from Bio import PDB
from Bio.PDB import Superimposer
from Bio.PDB import DSSP
from Bio.PDB.DSSP import dssp_dict_from_pdb_file

from tmtools.io import get_structure, get_residue_data
from tmtools import tm_align

import freesasa

from mpl_toolkits.mplot3d import Axes3D
from scipy.stats import kendalltau

In [ ]:
amino_acids = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y','_']

def translate_codon(codon):
    genetic_code = {
        'ATA':'I', 'ATC':'I', 'ATT':'I', 'ATG':'M',
        'ACA':'T', 'ACC':'T', 'ACG':'T', 'ACT':'T',
        'AAC':'N', 'AAT':'N', 'AAA':'K', 'AAG':'K',
        'AGC':'S', 'AGT':'S', 'AGA':'R', 'AGG':'R',
        'CTA':'L', 'CTC':'L', 'CTG':'L', 'CTT':'L',
        'CCA':'P', 'CCC':'P', 'CCG':'P', 'CCT':'P',
        'CAC':'H', 'CAT':'H', 'CAA':'Q', 'CAG':'Q',
        'CGA':'R', 'CGC':'R', 'CGG':'R', 'CGT':'R',
        'GTA':'V', 'GTC':'V', 'GTG':'V', 'GTT':'V',
        'GCA':'A', 'GCC':'A', 'GCG':'A', 'GCT':'A',
        'GAC':'D', 'GAT':'D', 'GAA':'E', 'GAG':'E',
        'GGA':'G', 'GGC':'G', 'GGG':'G', 'GGT':'G',
        'TCA':'S', 'TCC':'S', 'TCG':'S', 'TCT':'S',
        'TTC':'F', 'TTT':'F', 'TTA':'L', 'TTG':'L',
        'TAC':'Y', 'TAT':'Y', 'TAA':'_', 'TAG':'_',
        'TGC':'C', 'TGT':'C', 'TGA':'_', 'TGG':'W',
    }
    
    return genetic_code.get(codon.upper(), 'X')  # 'X' represents an unknown codon

## Read from and write to FASTA files

In [ ]:
def read_fasta(file_path):
    sequences = {}
    with open(file_path, 'r') as file:
        current_id = None
        current_sequence = ""
        for line in file:
            line = line.strip()
            if line.startswith('>'):
                if current_id is not None:
                    sequences[current_id] = current_sequence
                current_id = line[1:]
                current_sequence = ""
            else:
                current_sequence += line
        if current_id is not None:
            sequences[current_id] = current_sequence
    return sequences

def write_fasta(file_path, sequences):
    with open(file_path, 'w') as file:
        for identifier, sequence in sequences.items():
            file.write('>' + identifier + '\n')
            file.write(sequence + '\n')

## Extract the corresponding nucleotide sequence of the ACE2 receptor binding domain

NCBI Reference Sequence: NC_045512.2

In [ ]:
def extract_and_save_substring(input_file, output_file, start_pos, end_pos):
    sequences = read_fasta(input_file)
    extracted_sequences = {}
    
    for identifier, sequence in sequences.items():
        extracted_substring = sequence[start_pos-1:end_pos]  # Adjusting positions to 0-based index
        extracted_sequences[identifier] = extracted_substring

    write_fasta(output_file, extracted_sequences)
    print("Extraction complete. Substrings saved to", output_file)

# Example usage:
input_file = "NC_045512.fasta"
output_file = "extracted_substrings.fasta"
start_pos = 21563
end_pos = 25384

extract_and_save_substring(input_file, output_file, start_pos, end_pos)


## Plot spoke wheel of number of codons that can mutate to each of the 20 amino acids

In [ ]:
exclude_aa = '_'

# Filter out the excluded amino acid
filtered_amino_acids = [aa for aa in amino_acids if aa != exclude_aa]
filtered_total_counts = {aa: count for aa, count in total_counts.items() if aa != exclude_aa}

#phase_transitions = [80, 80, 75, 110, 100, 110, 100, 110, 110, 80, 80, 75, 110, 90, 100, 75, 100, 90, 100, 90]

fig = go.Figure(data=go.Scatterpolar(
  r=[filtered_total_counts[aa] for aa in filtered_amino_acids],
  theta=filtered_amino_acids,
  fill='toself',
  name = 'Max Amino Acid Occurence',
    line=dict(color='blue')
))

fig.add_trace(go.Scatterpolar(
      r=phase_transitions,
      theta=filtered_amino_acids,
      fill='toself',
      name='Phase Transitions',
    marker=dict(color='orange') 
))


fig.update_layout(
  polar=dict(
    radialaxis=dict(
            visible = True,
            range=[0, 195]
        ),
  ),
  showlegend=True
)

fig.show()

## Permutated Sequences

In [ ]:


def generate_random_sequence_with_same_distribution(original_sequence):
    # Count occurrences of each amino acid in the original sequence
    amino_acid_counts = {aa: original_sequence.count(aa) for aa in set(original_sequence)}
    
    # Generate a random sequence with the same distribution of amino acids
    random_sequence = ""
    for aa, count in amino_acid_counts.items():
        random_sequence += aa * count
    
    random_sequence = ''.join(random.sample(random_sequence, len(random_sequence)))
    
    return random_sequence

# Read the original sequence from a FASTA file
def read_sequence_from_fasta(file_path):
    for record in SeqIO.parse(file_path, "fasta"):
        return str(record.seq)

# Create a folder to save random sequences if it doesn't exist
def create_folder(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

# Example: Path to the FASTA file
file_path = "variant_analysis/extracted_substrings_OUT.fasta"
# Example: Path to the folder to save random sequences
output_folder = "control_seqs/input"

# Read the original sequence from the FASTA file
original_sequence = read_sequence_from_fasta(file_path)
print("Original Sequence:", original_sequence)

# Create the output folder if it doesn't exist
create_folder(output_folder)

# Generate 10 random sequences with the same distribution of amino acids as the original sequence
for i in range(1, 11):
    random_sequence = generate_random_sequence_with_same_distribution(original_sequence)
    output_file_path = os.path.join(output_folder, f"random_sequence_{i}.fasta")
    with open(output_file_path, "w") as output_file:
        output_file.write(f">Random_Sequence_{i}\n{random_sequence}")

print("Random sequences saved successfully.")


### Check if distribution of amino acids is the same

In [ ]:

def read_fasta(file_path):
    sequences = []
    with open(file_path, 'r') as file:
        sequence = ''
        for line in file:
            line = line.strip()
            if line.startswith('>'):
                if sequence:
                    sequences.append(sequence)
                    sequence = ''
            else:
                sequence += line
        if sequence:
            sequences.append(sequence)
    return sequences

def count_amino_acids(sequences):
    amino_acid_counts = []
    for sequence in sequences:
        counts = Counter(sequence)
        amino_acid_counts.append(counts)
    return amino_acid_counts

def check_amino_acid_distribution(folder_path):
    amino_acid_distributions = []
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.fasta'):
            file_path = os.path.join(folder_path, file_name)
            sequences = read_fasta(file_path)
            amino_acid_counts = count_amino_acids(sequences)
            amino_acid_distributions.extend(amino_acid_counts)

    reference_distribution = amino_acid_distributions[0]
    for distribution in amino_acid_distributions[1:]:
        if distribution != reference_distribution:
            return False

    return True

# Example usage
output_folder = "control_seqs/input/"

if check_amino_acid_distribution(output_folder):
    print('true')
else:
    print('false')


## Prints amino acids each codon can mutate to

In [ ]:

def translate_codon(codon):
    codon_table = {
        'ATA':'I', 'ATC':'I', 'ATT':'I', 'ATG':'M',
        'ACA':'T', 'ACC':'T', 'ACG':'T', 'ACT':'T',
        'AAC':'N', 'AAT':'N', 'AAA':'K', 'AAG':'K',
        'AGC':'S', 'AGT':'S', 'AGA':'R', 'AGG':'R',
        'CTA':'L', 'CTC':'L', 'CTG':'L', 'CTT':'L',
        'CCA':'P', 'CCC':'P', 'CCG':'P', 'CCT':'P',
        'CAC':'H', 'CAT':'H', 'CAA':'Q', 'CAG':'Q',
        'CGA':'R', 'CGC':'R', 'CGG':'R', 'CGT':'R',
        'GTA':'V', 'GTC':'V', 'GTG':'V', 'GTT':'V',
        'GCA':'A', 'GCC':'A', 'GCG':'A', 'GCT':'A',
        'GAC':'D', 'GAT':'D', 'GAA':'E', 'GAG':'E',
        'GGA':'G', 'GGC':'G', 'GGG':'G', 'GGT':'G',
        'TCA':'S', 'TCC':'S', 'TCG':'S', 'TCT':'S',
        'TTC':'F', 'TTT':'F', 'TTA':'L', 'TTG':'L',
        'TAC':'Y', 'TAT':'Y', 'TAA':'_', 'TAG':'_',
        'TGC':'C', 'TGT':'C', 'TGA':'_', 'TGG':'W',
    }
    return codon_table.get(codon, 'X')

def count_missense_mutations(codon):
    genetic_code = {
        'A': ['G', 'C', 'T'],
        'C': ['G', 'A', 'T'],
        'G': ['A', 'C', 'T'],
        'T': ['A', 'C', 'G']
    }
    
    original_amino_acid = translate_codon(codon)
    mutations = set()
    mutated_amino_acids = set()
    
    for i in range(3):
        original_nucleotide = codon[i]
        for mutation in genetic_code[original_nucleotide]:
            if mutation != original_nucleotide:
                mutated_codon = codon[:i] + mutation + codon[i+1:]
                mutated_amino_acid = translate_codon(mutated_codon)
                if mutated_amino_acid != original_amino_acid:
                    mutations.add(mutated_amino_acid)
                    mutated_amino_acids.add(mutated_amino_acid)
    
    return len(mutations), mutated_amino_acids

def count_missense_mutations_for_sequence(sequence):
    mutations_counts = []
    
    for i in range(0, len(sequence) - 2, 3):
        codon = sequence[i:i+3]
        mutations, mutated_amino_acids = count_missense_mutations(codon)
        mutations_counts.append((mutations, mutated_amino_acids))
    
    return mutations_counts

def read_fasta(file_path):
    sequences = {}
    with open(file_path, 'r') as file:
        sequence_id = None
        sequence = ''
        for line in file:
            if line.startswith('>'):
                if sequence_id is not None:
                    sequences[sequence_id] = sequence
                sequence_id = line.strip()
                sequence = ''
            else:
                sequence += line.strip()
        if sequence_id is not None:
            sequences[sequence_id] = sequence
    return sequences

# Read nucleotide sequence from file
file_path = "variant_analysis/ACE2-RBD_nucleotides.fasta"
sequences = read_fasta(file_path)

# Perform experiment for each sequence
for identifier, sequence in sequences.items():
    print("Sequence with identifier:", identifier)
    mutations_counts = count_missense_mutations_for_sequence(sequence)
    for i, (mutations, mutated_amino_acids) in enumerate(mutations_counts, start=1):
        print(f"Codon {i} ({sequence[(i-1)*3:i*3]}) can mutate to {mutations} unique amino acids: {mutated_amino_acids}")


### Counts number of mutations to each amino acid

In [ ]:

def translate_codon(codon):
    codon_table = {
        'ATA': 'I', 'ATC': 'I', 'ATT': 'I', 'ATG': 'M',
        'ACA': 'T', 'ACC': 'T', 'ACG': 'T', 'ACT': 'T',
        'AAC': 'N', 'AAT': 'N', 'AAA': 'K', 'AAG': 'K',
        'AGC': 'S', 'AGT': 'S', 'AGA': 'R', 'AGG': 'R',
        'CTA': 'L', 'CTC': 'L', 'CTG': 'L', 'CTT': 'L',
        'CCA': 'P', 'CCC': 'P', 'CCG': 'P', 'CCT': 'P',
        'CAC': 'H', 'CAT': 'H', 'CAA': 'Q', 'CAG': 'Q',
        'CGA': 'R', 'CGC': 'R', 'CGG': 'R', 'CGT': 'R',
        'GTA': 'V', 'GTC': 'V', 'GTG': 'V', 'GTT': 'V',
        'GCA': 'A', 'GCC': 'A', 'GCG': 'A', 'GCT': 'A',
        'GAC': 'D', 'GAT': 'D', 'GAA': 'E', 'GAG': 'E',
        'GGA': 'G', 'GGC': 'G', 'GGG': 'G', 'GGT': 'G',
        'TCA': 'S', 'TCC': 'S', 'TCG': 'S', 'TCT': 'S',
        'TTC': 'F', 'TTT': 'F', 'TTA': 'L', 'TTG': 'L',
        'TAC': 'Y', 'TAT': 'Y', 'TAA': '_', 'TAG': '_',
        'TGC': 'C', 'TGT': 'C', 'TGA': '_', 'TGG': 'W',
    }
    return codon_table.get(codon, 'X')

def count_missense_mutations(codon):
    genetic_code = {
        'A': ['G', 'C', 'T'],
        'C': ['G', 'A', 'T'],
        'G': ['A', 'C', 'T'],
        'T': ['A', 'C', 'G']
    }
    
    original_amino_acid = translate_codon(codon)
    mutations = set()
    mutated_amino_acids = set()
    
    for i in range(3):
        original_nucleotide = codon[i]
        for mutation in genetic_code[original_nucleotide]:
            if mutation != original_nucleotide:
                mutated_codon = codon[:i] + mutation + codon[i+1:]
                mutated_amino_acid = translate_codon(mutated_codon)
                if mutated_amino_acid != original_amino_acid:
                    mutations.add(mutated_amino_acid)
                    mutated_amino_acids.add(mutated_amino_acid)
    
    return len(mutations), mutated_amino_acids

def count_missense_mutations_for_sequence(sequence):
    amino_acid_mutation_count = Counter()
    
    for i in range(0, len(sequence) - 2, 3):
        codon = sequence[i:i+3]
        mutations, mutated_amino_acids = count_missense_mutations(codon)
        amino_acid_mutation_count.update(mutated_amino_acids)
    
    return amino_acid_mutation_count

def read_fasta(file_path):
    sequences = {}
    with open(file_path, 'r') as file:
        sequence_id = None
        sequence = ''
        for line in file:
            if line.startswith('>'):
                if sequence_id is not None:
                    sequences[sequence_id] = sequence
                sequence_id = line.strip()
                sequence = ''
            else:
                sequence += line.strip()
        if sequence_id is not None:
            sequences[sequence_id] = sequence
    return sequences

# Read nucleotide sequence from file
file_path = "variant_analysis/ACE2-RBD_nucleotides.fasta"
sequences = read_fasta(file_path)

# Perform experiment for each sequence
for identifier, sequence in sequences.items():
    print(f"Sequence with identifier: {identifier}")
    amino_acid_mutation_count = count_missense_mutations_for_sequence(sequence)
    for amino_acid, count in amino_acid_mutation_count.items():
        print(f"Amino acid {amino_acid} can be formed by missense mutations {count} times")


In [ ]:


def translate_codon(codon):
    codon_table = {
        'ATA': 'I', 'ATC': 'I', 'ATT': 'I', 'ATG': 'M',
        'ACA': 'T', 'ACC': 'T', 'ACG': 'T', 'ACT': 'T',
        'AAC': 'N', 'AAT': 'N', 'AAA': 'K', 'AAG': 'K',
        'AGC': 'S', 'AGT': 'S', 'AGA': 'R', 'AGG': 'R',
        'CTA': 'L', 'CTC': 'L', 'CTG': 'L', 'CTT': 'L',
        'CCA': 'P', 'CCC': 'P', 'CCG': 'P', 'CCT': 'P',
        'CAC': 'H', 'CAT': 'H', 'CAA': 'Q', 'CAG': 'Q',
        'CGA': 'R', 'CGC': 'R', 'CGG': 'R', 'CGT': 'R',
        'GTA': 'V', 'GTC': 'V', 'GTG': 'V', 'GTT': 'V',
        'GCA': 'A', 'GCC': 'A', 'GCG': 'A', 'GCT': 'A',
        'GAC': 'D', 'GAT': 'D', 'GAA': 'E', 'GAG': 'E',
        'GGA': 'G', 'GGC': 'G', 'GGG': 'G', 'GGT': 'G',
        'TCA': 'S', 'TCC': 'S', 'TCG': 'S', 'TCT': 'S',
        'TTC': 'F', 'TTT': 'F', 'TTA': 'L', 'TTG': 'L',
        'TAC': 'Y', 'TAT': 'Y', 'TAA': '_', 'TAG': '_',
        'TGC': 'C', 'TGT': 'C', 'TGA': '_', 'TGG': 'W',
    }
    return codon_table.get(codon, 'X')

def count_missense_mutations(codon):
    genetic_code = {
        'A': ['G', 'C', 'T'],
        'C': ['G', 'A', 'T'],
        'G': ['A', 'C', 'T'],
        'T': ['A', 'C', 'G']
    }
    
    original_amino_acid = translate_codon(codon)
    mutations = set()
    mutated_amino_acids = set()
    
    for i in range(3):
        original_nucleotide = codon[i]
        for mutation in genetic_code[original_nucleotide]:
            if mutation != original_nucleotide:
                mutated_codon = codon[:i] + mutation + codon[i+1:]
                mutated_amino_acid = translate_codon(mutated_codon)
                if mutated_amino_acid != original_amino_acid:
                    mutations.add(mutated_amino_acid)
                    mutated_amino_acids.add(mutated_amino_acid)
    
    return len(mutations), mutated_amino_acids

def count_missense_mutations_for_sequence(sequence):
    amino_acid_mutation_count = Counter()
    
    for i in range(0, len(sequence) - 2, 3):
        codon = sequence[i:i+3]
        mutations, mutated_amino_acids = count_missense_mutations(codon)
        amino_acid_mutation_count.update(mutated_amino_acids)
    
    return amino_acid_mutation_count

def read_fasta(file_path):
    sequences = {}
    with open(file_path, 'r') as file:
        sequence_id = None
        sequence = ''
        for line in file:
            if line.startswith('>'):
                if sequence_id is not None:
                    sequences[sequence_id] = sequence
                sequence_id = line.strip()
                sequence = ''
            else:
                sequence += line.strip()
        if sequence_id is not None:
            sequences[sequence_id] = sequence
    return sequences

# Read nucleotide sequence from file
file_path = "variant_analysis/ACE2-RBD_nucleotides.fasta"
sequences = read_fasta(file_path)

# Aggregate missense mutation counts across all sequences
amino_acids = 'ACDEFGHIKLMNPQRSTVWY_'
total_counts = {aa: 0 for aa in amino_acids}
for identifier, sequence in sequences.items():
    print("Sequence with identifier:", identifier)
    mutation_counts = count_missense_mutations_for_sequence(sequence)
    for aa, count in mutation_counts.items():
        total_counts[aa] += count

# Plotting the results
fig, ax = plt.subplots(figsize=(10, 6))

x = np.arange(len(amino_acids))

# Plot bars for count
ax.bar(x, [total_counts[aa] for aa in amino_acids], color='skyblue')
ax.set_xticks(x)
ax.set_xticklabels(amino_acids)
ax.set_xlabel('Amino Acid')
ax.set_ylabel('Count of Missense Mutations')
ax.set_title('Count of Missense Mutations to Each Amino Acid')

plt.show()


In [ ]:
exclude_aa = '_'

# Filter out the excluded amino acid
filtered_amino_acids = [aa for aa in amino_acids if aa != exclude_aa]
filtered_total_counts = {aa: count for aa, count in total_counts.items() if aa != exclude_aa}

# Manually set the amino acid counts
amino_acid_counts = {
    'A': 93, 'C': 61, 'D': 77, 'E': 43, 'F': 73, 'G': 79, 'H': 60, 'I': 86,
    'K': 50, 'L': 74, 'M': 18, 'N': 66, 'P': 62, 'Q': 28, 'R': 74, 'S': 136,
    'T': 90, 'V': 92, 'W': 13, 'Y': 73
}

phase_transitions = [80, 80, 75, 110, 100, 110, 100, 110, 110, 80, 80, 75, 110, 90, 100, 75, 100, 90, 100, 90]

fig = go.Figure(data=go.Scatterpolar(
    r=[amino_acid_counts[aa] for aa in filtered_amino_acids],
    theta=filtered_amino_acids,
    fill='toself',
    name='Max Amino Acid Occurrence',
    line=dict(color='blue')
))

"""
fig.add_trace(go.Scatterpolar(
      r=phase_transitions,
      theta=filtered_amino_acids,
      fill='toself',
      name='Phase Transitions',
    marker=dict(color='orange') 
))
"""

fig.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, 195]
        ),
    ),
    showlegend=True
)

fig.show()


# Trajectories with genetic constraint

In [ ]:


# Amino acid to codon mapping (considering only standard codons for simplicity)
codon_table = {
    'A': ['GCT', 'GCC', 'GCA', 'GCG'],
    'C': ['TGT', 'TGC'],
    'D': ['GAT', 'GAC'],
    'E': ['GAA', 'GAG'],
    'F': ['TTT', 'TTC'],
    'G': ['GGT', 'GGC', 'GGA', 'GGG'],
    'H': ['CAT', 'CAC'],
    'I': ['ATT', 'ATC', 'ATA'],
    'K': ['AAA', 'AAG'],
    'L': ['TTA', 'TTG', 'CTT', 'CTC', 'CTA', 'CTG'],
    'M': ['ATG'],
    'N': ['AAT', 'AAC'],
    'P': ['CCT', 'CCC', 'CCA', 'CCG'],
    'Q': ['CAA', 'CAG'],
    'R': ['CGT', 'CGC', 'CGA', 'CGG', 'AGA', 'AGG'],
    'S': ['TCT', 'TCC', 'TCA', 'TCG', 'AGT', 'AGC'],
    'T': ['ACT', 'ACC', 'ACA', 'ACG'],
    'V': ['GTT', 'GTC', 'GTA', 'GTG'],
    'W': ['TGG'],
    'Y': ['TAT', 'TAC'],
    '*': ['TAA', 'TAG', 'TGA'],  # Stop codons
}

# Reverse codon table to get amino acid for a codon
reverse_codon_table = {codon: aa for aa, codons in codon_table.items() for codon in codons}

# Function to translate a nucleotide sequence to an amino acid sequence
def translate(nucleotide_sequence):
    return ''.join(reverse_codon_table[nucleotide_sequence[i:i+3]] for i in range(0, len(nucleotide_sequence), 3))

# Function to check if a codon can be mutated to one of the target amino acid codons through a missense mutation
def can_mutate_to_target(codon, target_amino_acid):
    target_codons = codon_table[target_amino_acid]
    for target_codon in target_codons:
        if sum(1 for i in range(3) if codon[i] != target_codon[i]) == 1:
            return True
    return False

# Function to mutate a codon to one of the target amino acid codons
def mutate_codon_to_target(codon, target_amino_acid):
    target_codons = codon_table[target_amino_acid]
    for target_codon in target_codons:
        if sum(1 for i in range(3) if codon[i] != target_codon[i]) == 1:
            return target_codon
    return codon

# Function to mutate nucleotide sequence towards homopolymer amino acid sequence
def mutate_to_homopolymer(nucleotide_sequence, target_amino_acid, output_folder, max_attempts=1000):
    amino_acid_sequence = translate(nucleotide_sequence)
    step = 0
    attempts = 0
    
    while set(amino_acid_sequence) != {target_amino_acid} and attempts < max_attempts:
        codon_index = random.randint(0, len(nucleotide_sequence) // 3 - 1)
        codon_start = codon_index * 3
        current_codon = nucleotide_sequence[codon_start:codon_start+3]
        
        if translate(current_codon) != target_amino_acid and can_mutate_to_target(current_codon, target_amino_acid):
            new_codon = mutate_codon_to_target(current_codon, target_amino_acid)
            nucleotide_sequence = nucleotide_sequence[:codon_start] + new_codon + nucleotide_sequence[codon_start+3:]
            amino_acid_sequence = translate(nucleotide_sequence)
            step += 1
            output_file = os.path.join(output_folder, f"{target_amino_acid}_trajectory_{step}.fasta")
            with open(output_file, 'w') as f:
                f.write(f">{target_amino_acid}_trajectory_{step}\n{amino_acid_sequence}\n")
            attempts = 0  # Reset attempts counter after a successful mutation
        else:
            attempts += 1  # Increment attempts counter if no successful mutation
    
    if attempts >= max_attempts:
        print(f"Reached maximum attempts ({max_attempts}) for {target_amino_acid}. Moving to next amino acid.")
    
    return nucleotide_sequence

# Function to read nucleotide sequence from a FASTA file
def read_fasta(file_path):
    for record in SeqIO.parse(file_path, "fasta"):
        return str(record.seq)

# Main function to process the nucleotide sequence
def process_nucleotide_sequence(file_path, output_folder):
    initial_nucleotide_sequence = read_fasta(file_path)
    
    # Ensure the sequence length is 585 nucleotides (195 codons)
    if len(initial_nucleotide_sequence) != 585:
        raise ValueError("The nucleotide sequence must be 585 nucleotides long.")
    
    # Create output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Iterate over each amino acid to create homopolymer trajectories
    for amino_acid in codon_table.keys():
        if amino_acid == '*':  # Skip stop codons
            continue
        print(f"Starting mutation towards {amino_acid} homopolymer")
        mutate_to_homopolymer(initial_nucleotide_sequence, amino_acid, output_folder)
        print(f"Final {amino_acid} homopolymer sequence saved to {output_folder}\n")

# Example usage
fasta_file_path = "variant_analysis/ACE2-RBD_nucleotides.fasta"
output_folder = "variant_analysis/TEST/"
process_nucleotide_sequence(fasta_file_path, output_folder)


### sort into batches of 100 to prepare for AlphaFold2 prediction

In [ ]:
import os
import shutil

def organize_into_subfolders(output_folder, subfolder_size=100):
    # List all files in the output folder
    files = sorted([f for f in os.listdir(output_folder) if os.path.isfile(os.path.join(output_folder, f))])
    
    # Create subfolders and move files
    subfolder_index = 1
    for i in range(0, len(files), subfolder_size):
        subfolder_name = f"sub{subfolder_index}"
        subfolder_path = os.path.join(output_folder, subfolder_name)
        
        # Create subfolder if it doesn't exist
        if not os.path.exists(subfolder_path):
            os.makedirs(subfolder_path)
        
        # Move files to the subfolder
        for file in files[i:i + subfolder_size]:
            shutil.move(os.path.join(output_folder, file), os.path.join(subfolder_path, file))
        
        subfolder_index += 1

# Example usage
output_folder = "variant_analysis/genetic_variation_fasta/"
organize_into_subfolders(output_folder)


# define functions

In [ ]:



def load_structures(input_path, ref_path):
    p = PDBParser(QUIET=True)
    struct_path = Path(input_path)
    ref_path = Path(ref_path)
    parent_dir = struct_path.parent
    key_name = parent_dir.name
    fixed_struct = p.get_structure('fixed', ref_path)
    moving_struct = p.get_structure('moving', struct_path)

    return fixed_struct, moving_struct, key_name

def calculate_rmsd(fixed_atoms, moving_atoms, structure):
    super_imposer = Superimposer()
    super_imposer.set_atoms(fixed_atoms, moving_atoms)
    super_imposer.apply(structure.get_atoms())
    return super_imposer.rms

def rmsd(input_path, ref_path):
    fixed_struct, moving_struct, key_name = load_structures(input_path, ref_path)
    fixed_model = fixed_struct[0]
    for chain in fixed_model:
        if chain.id == 'E':
            all_atoms_fixed = [atom for residue in chain.get_residues() for atom in residue]
            all_atoms_moving = [atom for residue in moving_struct.get_residues() for atom in residue]
            if len(all_atoms_fixed) > len(all_atoms_moving):
                dif = len(all_atoms_fixed) - len(all_atoms_moving)
                all_atoms_fixed = all_atoms_fixed[:len(all_atoms_fixed) - dif]
            else:
                dif = len(all_atoms_moving) - len(all_atoms_fixed)
                all_atoms_moving = all_atoms_moving[:len(all_atoms_moving) - dif]
            assert len(all_atoms_fixed) == len(all_atoms_moving)
            rmsd_value = calculate_rmsd(all_atoms_fixed, all_atoms_moving, chain)
            return rmsd_value

    return None

def tm_score(input_path, ref_path):
    ref_struct, query_struct, key_name = load_structures(input_path, ref_path)
    chain = next(ref_struct.get_chains())
    ref_coords, ref_seq = get_residue_data(chain)
    chain = next(query_struct.get_chains())
    coords, seq = get_residue_data(chain)
    res = tm_align(ref_coords, coords, ref_seq, seq)
    tm_score = res.tm_norm_chain1
    return tm_score

def sasa(input_path):
    structure = freesasa.Structure(str(input_path))
    result = freesasa.calc(structure)
    total_sasa = result.totalArea()
    return total_sasa


input_path = 'variant_analysis/pdb_metrics/A_trajectory_1_unrelaxed_rank_001_alphafold2_ptm_model_3_seed_000.pdb'
ref_path = 'ACE2(1).pdb'

## 3D plot of permutated sequences

In [ ]:


# Function to extract PLDDT values from a PDB file
def extract_plddt(pdb_file):
    plddts = []
    with open(pdb_file, 'r') as file:
        for line in file:
            if line.startswith('ATOM') and line[13:15] == 'CA':
                plddt = float(line[60:66].strip())
                plddts.append(plddt)
    return plddts

# Function to calculate the average PLDDT
def calculate_average_plddt(pdb_file):
    plddts = extract_plddt(pdb_file)
    average_plddt = np.mean(plddts)
    return average_plddt

# Function to load structures
def load_structures(input_path, ref_path):
    p = PDBParser(QUIET=True)
    struct_path = Path(input_path)
    ref_path = Path(ref_path)
    parent_dir = struct_path.parent
    key_name = parent_dir.name
    fixed_struct = p.get_structure('fixed', ref_path)
    moving_struct = p.get_structure('moving', struct_path)

    return fixed_struct, moving_struct, key_name

# Function to calculate RMSD
def calculate_rmsd(fixed_atoms, moving_atoms, structure):
    super_imposer = Superimposer()
    super_imposer.set_atoms(fixed_atoms, moving_atoms)
    super_imposer.apply(structure.get_atoms())
    return super_imposer.rms

# Function to get RMSD
def rmsd(input_path, ref_path):
    fixed_struct, moving_struct, key_name = load_structures(input_path, ref_path)
    fixed_model = fixed_struct[0]
    for chain in fixed_model:
        if chain.id == 'E':
            all_atoms_fixed = [atom for residue in chain.get_residues() for atom in residue]
            all_atoms_moving = [atom for residue in moving_struct.get_residues() for atom in residue]
            if len(all_atoms_fixed) > len(all_atoms_moving):
                dif = len(all_atoms_fixed) - len(all_atoms_moving)
                all_atoms_fixed = all_atoms_fixed[:len(all_atoms_fixed) - dif]
            else:
                dif = len(all_atoms_moving) - len(all_atoms_fixed)
                all_atoms_moving = all_atoms_moving[:len(all_atoms_moving) - dif]
            assert len(all_atoms_fixed) == len(all_atoms_moving)
            rmsd_value = calculate_rmsd(all_atoms_fixed, all_atoms_moving, chain)
            return rmsd_value

    return None

# Function to get TM-score
def tm_score(input_path, ref_path):
    ref_struct, query_struct, key_name = load_structures(input_path, ref_path)
    chain = next(ref_struct.get_chains())
    ref_coords, ref_seq = get_residue_data(chain)
    chain = next(query_struct.get_chains())
    coords, seq = get_residue_data(chain)
    res = tm_align(ref_coords, coords, ref_seq, seq)
    tm_score = res.tm_norm_chain1
    return tm_score

def sasa(input_path):
    structure = freesasa.Structure(str(input_path))
    result = freesasa.calc(structure)
    total_sasa = result.totalArea()
    return total_sasa

# Function to process PDB files and compute RMSD, TM-score, and PLDDT
def process_structures(pdb_folder, ref_pdb):
    rmsd_values = []
    tm_scores = []
    plddts = []
    sequence_names = []

    for pdb_file in os.listdir(pdb_folder):
        if pdb_file.endswith('.pdb'):
            pdb_path = os.path.join(pdb_folder, pdb_file)
            plddt = calculate_average_plddt(pdb_path)
            rmsd_value = rmsd(pdb_path, ref_pdb)
            tm_score_value = tm_score(pdb_path, ref_pdb)

            plddts.append(plddt)
            rmsd_values.append(rmsd_value)
            tm_scores.append(tm_score_value)
            sequence_names.append(pdb_file)

    return plddts, rmsd_values, tm_scores, sequence_names

# Function to create a 3D plot
def plot_3d(plddts, rmsd_values, tm_scores, sequence_names, ref_plddt, ref_rmsd, ref_tm_score):
    fig = plt.figure(figsize=(12, 8))
    ax = fig.add_subplot(111, projection='3d')

    # Plotting the sequences
    sc = ax.scatter(plddts, rmsd_values, tm_scores, c='b', label='Random Permutations of Reference Sequence')
    ax.scatter([ref_plddt], [ref_rmsd], [ref_tm_score], c='r', label='Reference Sequence', marker='*', s=100)

    # Annotating the reference sequence
    #ax.text(ref_plddt, ref_rmsd, ref_tm_score, 'Reference Sequence', size=8, zorder=1, color='k')

    ax.set_xlabel('Average PLDDT')
    ax.set_ylabel('RMSD')
    ax.set_zlabel('TM-Score')
    ax.legend()
    #plt.title('3D Plot of PLDDT, RMSD, and TM-Score')
    plt.savefig('control_seqs/3D_plot.png')
    plt.show()

pdb_folder = 'control_seqs/pdb_control/'  # Update with your folder path
ref_pdb = 'ACE2(1).pdb' # Update with your reference PDB file path

# Processing the structures
plddts, rmsd_values, tm_scores, sequence_names = process_structures(pdb_folder, ref_pdb)

# Calculating metrics for the reference structure
ref_plddt = calculate_average_plddt(ref_pdb)
ref_rmsd = rmsd(ref_pdb, ref_pdb)  # RMSD of the reference to itself is 0
ref_tm_score = tm_score(ref_pdb, ref_pdb)  # TM-score of the reference to itself is 1

# Plotting the 3D scatter plot
plot_3d(plddts, rmsd_values, tm_scores, sequence_names, ref_plddt, ref_rmsd, ref_tm_score)


## Kendalls Tau of permutated sequences

In [ ]:


def read_fasta_sequence(file_path):
    """Read the first sequence from a FASTA file."""
    with open(file_path, 'r') as file:
        record = next(SeqIO.parse(file, "fasta"))
        return str(record.seq)

def calculate_kendall_tau_distance(original_seq, permuted_seq):
    if len(original_seq) != len(permuted_seq):
        raise ValueError("Sequences must be of the same length")
    
    # Create rank lists for the original and permuted sequences
    original_ranks = list(range(len(original_seq)))
    permuted_ranks = [original_seq.index(aa) for aa in permuted_seq]
    
    # Calculate Kendall tau distance
    tau, _ = kendalltau(original_ranks, permuted_ranks)
    return tau

def process_sequences(directory, original_sequence_file):
    original_sequence = read_fasta_sequence(original_sequence_file)
    results = []

    for file_name in os.listdir(directory):
        file_path = os.path.join(directory, file_name)
        if os.path.isfile(file_path) and file_name.endswith('.fasta'):
            permuted_sequence = read_fasta_sequence(file_path)
            tau = calculate_kendall_tau_distance(original_sequence, permuted_sequence)
            results.append((file_name, tau))
    
    return results

def print_results_table(results):
    df = pd.DataFrame(results, columns=['Sequence File', 'Kendall Tau Distance'])
    print(df.to_string(index=False))

# Define paths
directory = 'control_seqs/input/'
original_sequence_file = 'scriptsmax/sequence_datasets/2/SARS-CoV-ACE2.fasta'

# Process sequences and print results
results = process_sequences(directory, original_sequence_file)
print_results_table(results)


## Structural Metrics for permutated sequences

In [ ]:
def create_metrics_table(plddts, rmsd_values, tm_scores, sasas, sequence_names):
    data = {
        'Sequence': sequence_names,
        'PLDDT': plddts,
        'RMSD': rmsd_values,
        'TM-Score': tm_scores,
        'SASA': sasas
    }
    df = pd.DataFrame(data)
    return df

# Paths to the PDB files
#pdb_folder = 'control_seqs/pdb_control/'  # Update with your folder path
pdb_folder = 'control_seqs/pdb_control_alphafold/'  # Update with your folder path
ref_pdb = 'ACE2(1).pdb' # Update with your reference PDB file path

# Processing the structures
plddts, rmsd_values, tm_scores, sequence_names = process_structures(pdb_folder, ref_pdb)

# Calculating SASA values for all PDB files
sasas = [sasa(os.path.join(pdb_folder, pdb_file)) for pdb_file in os.listdir(pdb_folder) if pdb_file.endswith('.pdb')]

# Calculating metrics for the reference structure
ref_plddt = calculate_average_plddt(ref_pdb)
ref_rmsd = rmsd(ref_pdb, ref_pdb)  # RMSD of the reference to itself is 0
ref_tm_score = tm_score(ref_pdb, ref_pdb)  # TM-score of the reference to itself is 1
ref_sasa = sasa(ref_pdb)

# Create a table of metrics
metrics_table = create_metrics_table(plddts, rmsd_values, tm_scores, sasas, sequence_names)

# Output the table
print(metrics_table)


# pLDDT along trajectories

In [ ]:


def extract_plddt(pdb_file):
    plddts = []
    with open(pdb_file, 'r') as file:
        for line in file:
            if line.startswith('ATOM') and line[13:15] == 'CA':
                pldtt = float(line[60:66].strip())
                plddts.append(pldtt)
    return plddts

def calculate_average_plddt(pdb_file):
    plddts = extract_plddt(pdb_file)
    average_plddt = np.mean(plddts)
    return average_plddt

def analyze_trajectories(genetic_directories, theoretical_directories, smoothing_sigma=0.5):
    amino_acids = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']
    for amino_acid in amino_acids:
        plt.figure()
        plt.title(f'pLDDT Trajectory for Amino Acid {amino_acid}')
        plt.xlabel('k-step')
        plt.ylabel('Average pLDDT')
        plt.xlim(0, 195)  # Set x-axis limit
        plt.ylim(25, 95)  # Set y-axis limit

        all_plddts_genetic = []
        all_plddts_theoretical = []

        for parent_directory in genetic_directories:
            pdb_files = [f for f in os.listdir(parent_directory) if f.startswith(f'{amino_acid}') and f.endswith('.pdb')]
            pdb_files.sort(key=lambda x: int(re.search(r'\d+', x).group()))

            plddts = [calculate_average_plddt(os.path.join(parent_directory, pdb_file)) for pdb_file in pdb_files]
            all_plddts_genetic.append(plddts)

        for parent_directory in theoretical_directories:
            pdb_files = [f for f in os.listdir(parent_directory) if f.startswith(f'{amino_acid}') and f.endswith('.pdb')]
            pdb_files.sort(key=lambda x: int(re.search(r'\d+', x).group()))

            plddts = [calculate_average_plddt(os.path.join(parent_directory, pdb_file)) for pdb_file in pdb_files]
            all_plddts_theoretical.append(plddts)

        # Trim lists to the minimum length
        min_length = min(min(len(plddts) for plddts in all_plddts_genetic),
                         min(len(plddts) for plddts in all_plddts_theoretical))
        trimmed_plddts_genetic = [plddts[:min_length] for plddts in all_plddts_genetic]
        trimmed_plddts_theoretical = [plddts[:min_length] for plddts in all_plddts_theoretical]

        all_plddts_genetic = np.array(trimmed_plddts_genetic)
        all_plddts_theoretical = np.array(trimmed_plddts_theoretical)

        mean_plddts_genetic = np.mean(all_plddts_genetic, axis=0)
        std_plddts_genetic = np.std(all_plddts_genetic, axis=0)
        mean_plddts_theoretical = np.mean(all_plddts_theoretical, axis=0)
        std_plddts_theoretical = np.std(all_plddts_theoretical, axis=0)

        # Apply Gaussian smoothing
        smoothed_mean_genetic = gaussian_filter1d(mean_plddts_genetic, sigma=smoothing_sigma)
        smoothed_std_genetic = gaussian_filter1d(std_plddts_genetic, sigma=smoothing_sigma)
        smoothed_mean_theoretical = gaussian_filter1d(mean_plddts_theoretical, sigma=smoothing_sigma)
        smoothed_std_theoretical = gaussian_filter1d(std_plddts_theoretical, sigma=smoothing_sigma)

        x_range = range(len(smoothed_mean_genetic))
        plt.plot(x_range, smoothed_mean_genetic, label='pLDDT genetic sequences', linewidth=1)
        plt.fill_between(x_range, smoothed_mean_genetic - smoothed_std_genetic, smoothed_mean_genetic + smoothed_std_genetic, alpha=0.3)

        x_range = range(len(smoothed_mean_theoretical))
        plt.plot(x_range, smoothed_mean_theoretical, label='pLDDT theoretical sequences', linewidth=1, linestyle='--')
        plt.fill_between(x_range, smoothed_mean_theoretical - smoothed_std_theoretical, smoothed_mean_theoretical + smoothed_std_theoretical, alpha=0.3)

        plt.legend()

        plot_dir = 'variant_analysis/PLOTS/combined_theory_genetic/pLDDT/'
        if not os.path.exists(plot_dir):
            os.makedirs(plot_dir)

        plot_title = f'pLDDT_Trajectory_{amino_acid}'
        plot_filename = os.path.join(plot_dir, f'{plot_title}.png')
        plt.savefig(plot_filename)
        plt.show()

genetic_directories = ['variant_analysis/pdb_metrics/']
theoretical_directories = ['colabfold/metrics/pdb_1/', 'colabfold/metrics/pdb_2/']
analyze_trajectories(genetic_directories, theoretical_directories)


In [ ]:

def extract_plddt(pdb_file):
    plddts = []
    with open(pdb_file, 'r') as file:
        for line in file:
            if line.startswith('ATOM') and line[13:15] == 'CA':
                pldtt = float(line[60:66].strip())
                plddts.append(pldtt)
    return plddts

def calculate_average_plddt(pdb_file):
    plddts = extract_plddt(pdb_file)
    average_plddt = np.mean(plddts)
    return average_plddt

def analyze_trajectories(genetic_alphafold_directories, genetic_colabfold_directories, theoretical_directories, smoothing_sigma=0.5):
    amino_acids = ['A','P', 'S']
    for amino_acid in amino_acids:
        plt.figure()
        plt.title(f'pLDDT Trajectory for Amino Acid {amino_acid}')
        plt.xlabel('k-step')
        plt.ylabel('Average pLDDT')
        plt.xlim(0, 195)  # Set x-axis limit
        plt.ylim(25, 95)  # Set y-axis limit

        def process_directories(directories, label, linestyle, max_length=None):
            all_plddts = []

            for parent_directory in directories:
                pdb_files = [f for f in os.listdir(parent_directory) if f.startswith(f'{amino_acid}') and f.endswith('.pdb')]
                pdb_files.sort(key=lambda x: int(re.search(r'\d+', x).group()))

                plddts = [calculate_average_plddt(os.path.join(parent_directory, pdb_file)) for pdb_file in pdb_files]
                all_plddts.append(plddts)

            # Trim lists to the minimum length or to max_length if specified
            if max_length is not None:
                min_length = min(len(plddts) for plddts in all_plddts + [[0]*max_length])
                min_length = min(min_length, max_length)
            else:
                min_length = min(len(plddts) for plddts in all_plddts)
            
            trimmed_plddts = [plddts[:min_length] for plddts in all_plddts]

            all_plddts = np.array(trimmed_plddts)

            mean_plddts = np.mean(all_plddts, axis=0)
            std_plddts = np.std(all_plddts, axis=0)

            # Apply Gaussian smoothing
            smoothed_mean = gaussian_filter1d(mean_plddts, sigma=smoothing_sigma)
            smoothed_std = gaussian_filter1d(std_plddts, sigma=smoothing_sigma)

            x_range = range(len(smoothed_mean))
            plt.plot(x_range, smoothed_mean, label=label, linewidth=1, linestyle=linestyle)
            plt.fill_between(x_range, smoothed_mean - smoothed_std, smoothed_mean + smoothed_std, alpha=0.3)

        # Determine the minimum length of ColabFold sequences
        colabfold_lengths = []
        for parent_directory in genetic_colabfold_directories:
            pdb_files = [f for f in os.listdir(parent_directory) if f.startswith(f'{amino_acid}') and f.endswith('.pdb')]
            colabfold_lengths.append(len(pdb_files))
        min_colabfold_length = min(colabfold_lengths)

        # Process genetic sequences from AlphaFold and ColabFold
        
        process_directories(genetic_colabfold_directories, 'pLDDT genetic sequences (ColabFold)', '-')

        # Process theoretical sequences
        process_directories(theoretical_directories, 'pLDDT theoretical sequences', ':', max_length=min_colabfold_length)
        process_directories(genetic_alphafold_directories, 'pLDDT genetic sequences (AlphaFold)', '-', max_length=min_colabfold_length)

        plt.legend()

        plot_dir = 'variant_analysis/PLOTS/combined_theory_genetic_alphafold/pLDDT/'
        if not os.path.exists(plot_dir):
            os.makedirs(plot_dir)

        plot_title = f'pLDDT_Trajectory_{amino_acid}'
        plot_filename = os.path.join(plot_dir, f'{plot_title}.png')
        plt.savefig(plot_filename)
        plt.show()

# Example usage:
genetic_alphafold_directories = ['variant_analysis/pdb_metrics_alphafold_onlyPDB/']
genetic_colabfold_directories = ['variant_analysis/pdb_metrics/']
theoretical_directories = ['colabfold/metrics/pdb_1/', 'colabfold/metrics/pdb_2/']

analyze_trajectories(genetic_alphafold_directories, genetic_colabfold_directories, theoretical_directories)



# RMSD along trajectories

In [ ]:


def load_structures(input_path, ref_path):
    p = PDBParser(QUIET=True)
    struct_path = Path(input_path)
    ref_path = Path(ref_path)
    parent_dir = struct_path.parent
    key_name = parent_dir.name
    fixed_struct = p.get_structure('fixed', ref_path)
    moving_struct = p.get_structure('moving', struct_path)
    return fixed_struct, moving_struct, key_name

def calculate_rmsd(fixed_atoms, moving_atoms, structure):
    super_imposer = Superimposer()
    super_imposer.set_atoms(fixed_atoms, moving_atoms)
    super_imposer.apply(structure.get_atoms())
    return super_imposer.rms

def rmsd(input_path, ref_path):
    fixed_struct, moving_struct, key_name = load_structures(input_path, ref_path)
    fixed_model = fixed_struct[0]
    for chain in fixed_model:
        if chain.id == 'E':
            all_atoms_fixed = [atom for residue in chain.get_residues() for atom in residue]
            all_atoms_moving = [atom for residue in moving_struct.get_residues() for atom in residue]
            if len(all_atoms_fixed) > len(all_atoms_moving):
                dif = len(all_atoms_fixed) - len(all_atoms_moving)
                all_atoms_fixed = all_atoms_fixed[:len(all_atoms_fixed) - dif]
            else:
                dif = len(all_atoms_moving) - len(all_atoms_fixed)
                all_atoms_moving = all_atoms_moving[:len(all_atoms_moving) - dif]
            assert len(all_atoms_fixed) == len(all_atoms_moving)
            rmsd_value = calculate_rmsd(all_atoms_fixed, all_atoms_moving, chain)
            return rmsd_value
    return None

def analyze_rmsd_trajectories(genetic_directories, theoretical_directories, reference_pdb, smoothing_sigma=0.5):
    amino_acids = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']
    for amino_acid in amino_acids:
        plt.figure()
        plt.title(f'RMSD Trajectory for Amino Acid {amino_acid}')
        plt.xlabel('k-steps')
        plt.ylabel('RMSD')
        plt.xlim(0, 195)  # Set x-axis limit
        plt.ylim(0, 80)   # Adjust y-axis limit based on expected RMSD range

        all_rmsd_values_genetic = []
        all_rmsd_values_theoretical = []

        for parent_directory in genetic_directories:
            pdb_files = [f for f in os.listdir(parent_directory) if f.startswith(f'{amino_acid}') and f.endswith('.pdb')]
            pdb_files.sort(key=lambda x: int(re.search(r'\d+', x).group()))

            rmsd_values = [rmsd(os.path.join(parent_directory, pdb_file), reference_pdb) for pdb_file in pdb_files]
            all_rmsd_values_genetic.append(rmsd_values)

        for parent_directory in theoretical_directories:
            pdb_files = [f for f in os.listdir(parent_directory) if f.startswith(f'{amino_acid}') and f.endswith('.pdb')]
            pdb_files.sort(key=lambda x: int(re.search(r'\d+', x).group()))

            rmsd_values = [rmsd(os.path.join(parent_directory, pdb_file), reference_pdb) for pdb_file in pdb_files]
            all_rmsd_values_theoretical.append(rmsd_values)

        min_length_genetic = min(len(rmsd_values) for rmsd_values in all_rmsd_values_genetic)
        min_length_theoretical = min(len(rmsd_values) for rmsd_values in all_rmsd_values_theoretical)

        min_length = min(min_length_genetic, min_length_theoretical)

        trimmed_rmsd_values_genetic = [rmsd_values[:min_length] for rmsd_values in all_rmsd_values_genetic]
        trimmed_rmsd_values_theoretical = [rmsd_values[:min_length] for rmsd_values in all_rmsd_values_theoretical]

        all_rmsd_values_genetic = np.array(trimmed_rmsd_values_genetic)
        all_rmsd_values_theoretical = np.array(trimmed_rmsd_values_theoretical)

        mean_rmsd_genetic = np.mean(all_rmsd_values_genetic, axis=0)
        std_rmsd_genetic = np.std(all_rmsd_values_genetic, axis=0)

        mean_rmsd_theoretical = np.mean(all_rmsd_values_theoretical, axis=0)
        std_rmsd_theoretical = np.std(all_rmsd_values_theoretical, axis=0)

        # Apply Gaussian smoothing
        smoothed_mean_genetic = gaussian_filter1d(mean_rmsd_genetic, sigma=smoothing_sigma)
        smoothed_std_genetic = gaussian_filter1d(std_rmsd_genetic, sigma=smoothing_sigma)

        smoothed_mean_theoretical = gaussian_filter1d(mean_rmsd_theoretical, sigma=smoothing_sigma)
        smoothed_std_theoretical = gaussian_filter1d(std_rmsd_theoretical, sigma=smoothing_sigma)

        x_range = range(len(smoothed_mean_genetic))
        plt.plot(x_range, smoothed_mean_genetic, label='RMSD genetic sequences', linewidth=1)
        plt.fill_between(x_range, smoothed_mean_genetic - smoothed_std_genetic, smoothed_mean_genetic + smoothed_std_genetic, alpha=0.3)

        x_range_theoretical = range(len(smoothed_mean_theoretical))
        plt.plot(x_range_theoretical, smoothed_mean_theoretical, label='RMSD theoretical sequences', linewidth=1, linestyle='dashed')
        plt.fill_between(x_range_theoretical, smoothed_mean_theoretical - smoothed_std_theoretical, smoothed_mean_theoretical + smoothed_std_theoretical, alpha=0.3)

        plt.legend()

        plot_dir = 'variant_analysis/PLOTS/combined_theory_genetic/RMSD/'
        if not os.path.exists(plot_dir):
            os.makedirs(plot_dir)

        plot_title = f'RMSD_Trajectory_{amino_acid}'
        plot_filename = os.path.join(plot_dir, f'{plot_title}.png')
        plt.savefig(plot_filename)
        plt.show()

genetic_directories = ['variant_analysis/pdb_metrics/']
theoretical_directories = ['colabfold/metrics/pdb_1/', 'colabfold/metrics/pdb_2/']
reference_pdb = 'ACE2(1).pdb'
analyze_rmsd_trajectories(genetic_directories, theoretical_directories, reference_pdb)


In [ ]:


def load_structures(input_path, ref_path):
    p = PDBParser(QUIET=True)
    struct_path = Path(input_path)
    ref_path = Path(ref_path)
    parent_dir = struct_path.parent
    key_name = parent_dir.name
    fixed_struct = p.get_structure('fixed', ref_path)
    moving_struct = p.get_structure('moving', struct_path)
    return fixed_struct, moving_struct, key_name

def calculate_rmsd(fixed_atoms, moving_atoms, structure):
    super_imposer = Superimposer()
    super_imposer.set_atoms(fixed_atoms, moving_atoms)
    super_imposer.apply(structure.get_atoms())
    return super_imposer.rms

def rmsd(input_path, ref_path):
    fixed_struct, moving_struct, key_name = load_structures(input_path, ref_path)
    fixed_model = fixed_struct[0]
    for chain in fixed_model:
        if chain.id == 'E':
            all_atoms_fixed = [atom for residue in chain.get_residues() for atom in residue]
            all_atoms_moving = [atom for residue in moving_struct.get_residues() for atom in residue]
            if len(all_atoms_fixed) > len(all_atoms_moving):
                dif = len(all_atoms_fixed) - len(all_atoms_moving)
                all_atoms_fixed = all_atoms_fixed[:len(all_atoms_fixed) - dif]
            else:
                dif = len(all_atoms_moving) - len(all_atoms_fixed)
                all_atoms_moving = all_atoms_moving[:len(all_atoms_moving) - dif]
            assert len(all_atoms_fixed) == len(all_atoms_moving)
            rmsd_value = calculate_rmsd(all_atoms_fixed, all_atoms_moving, chain)
            return rmsd_value
    return None

def analyze_rmsd_trajectories(genetic_alphafold_directories, genetic_colabfold_directories, theoretical_directories, reference_pdb, smoothing_sigma=0.5):
    amino_acids = ['A','P', 'S']
    for amino_acid in amino_acids:
        plt.figure()
        plt.title(f'RMSD Trajectory for Amino Acid {amino_acid}')
        plt.xlabel('k-steps')
        plt.ylabel('RMSD')
        plt.xlim(0, 195)  # Set x-axis limit
        plt.ylim(0, 80)   # Adjust y-axis limit based on expected RMSD range

        def process_directories(directories, label, linestyle, max_length=None):
            all_rmsd_values = []

            for parent_directory in directories:
                pdb_files = [f for f in os.listdir(parent_directory) if f.startswith(f'{amino_acid}') and f.endswith('.pdb')]
                pdb_files.sort(key=lambda x: int(re.search(r'\d+', x).group()))

                rmsd_values = [rmsd(os.path.join(parent_directory, pdb_file), reference_pdb) for pdb_file in pdb_files]
                all_rmsd_values.append(rmsd_values)

            # Trim lists to the minimum length or to max_length if specified
            if max_length is not None:
                min_length = min(len(rmsd_values) for rmsd_values in all_rmsd_values + [[0]*max_length])
                min_length = min(min_length, max_length)
            else:
                min_length = min(len(rmsd_values) for rmsd_values in all_rmsd_values)
            
            trimmed_rmsd_values = [rmsd_values[:min_length] for rmsd_values in all_rmsd_values]

            all_rmsd_values = np.array(trimmed_rmsd_values)

            mean_rmsd = np.mean(all_rmsd_values, axis=0)
            std_rmsd = np.std(all_rmsd_values, axis=0)

            # Apply Gaussian smoothing
            smoothed_mean = gaussian_filter1d(mean_rmsd, sigma=smoothing_sigma)
            smoothed_std = gaussian_filter1d(std_rmsd, sigma=smoothing_sigma)

            x_range = range(len(smoothed_mean))
            plt.plot(x_range, smoothed_mean, label=label, linewidth=1, linestyle=linestyle)
            plt.fill_between(x_range, smoothed_mean - smoothed_std, smoothed_mean + smoothed_std, alpha=0.3)

        # Determine the minimum length of ColabFold sequences
        colabfold_lengths = []
        for parent_directory in genetic_colabfold_directories:
            pdb_files = [f for f in os.listdir(parent_directory) if f.startswith(f'{amino_acid}') and f.endswith('.pdb')]
            colabfold_lengths.append(len(pdb_files))
        min_colabfold_length = min(colabfold_lengths)

        # Process genetic sequences from AlphaFold and ColabFold
        
        process_directories(genetic_colabfold_directories, 'RMSD genetic sequences (ColabFold)', '-', max_length=min_colabfold_length)

        # Process theoretical sequences
        process_directories(theoretical_directories, 'RMSD theoretical sequences', ':', max_length=min_colabfold_length)
        
        process_directories(genetic_alphafold_directories, 'RMSD genetic sequences (AlphaFold)', '-', max_length=min_colabfold_length)

        plt.legend()

        plot_dir = 'variant_analysis/PLOTS/combined_theory_genetic_alphafold/RMSD/'
        if not os.path.exists(plot_dir):
            os.makedirs(plot_dir)

        plot_title = f'RMSD_Trajectory_{amino_acid}'
        plot_filename = os.path.join(plot_dir, f'{plot_title}.png')
        plt.savefig(plot_filename)
        plt.show()

# Example usage:
genetic_alphafold_directories = ['variant_analysis/pdb_metrics_alphafold_onlyPDB/']
genetic_colabfold_directories = ['variant_analysis/pdb_metrics/']
theoretical_directories = ['colabfold/metrics/pdb_1/', 'colabfold/metrics/pdb_2/']
reference_pdb = 'ACE2(1).pdb'

analyze_rmsd_trajectories(genetic_alphafold_directories, genetic_colabfold_directories, theoretical_directories, reference_pdb)


# TM Score along trajectories

In [ ]:

def load_structures(input_path, ref_path):
    p = PDBParser(QUIET=True)
    struct_path = Path(input_path)
    ref_path = Path(ref_path)
    parent_dir = struct_path.parent
    key_name = parent_dir.name
    fixed_struct = p.get_structure('fixed', ref_path)
    moving_struct = p.get_structure('moving', struct_path)
    return fixed_struct, moving_struct, key_name

def tm_score(input_path, ref_path):
    ref_struct, query_struct, key_name = load_structures(input_path, ref_path)
    chain = next(ref_struct.get_chains())
    ref_coords, ref_seq = get_residue_data(chain)
    chain = next(query_struct.get_chains())
    coords, seq = get_residue_data(chain)
    res = tm_align(ref_coords, coords, ref_seq, seq)
    if res is None or res.tm_norm_chain1 is None:
        print(f"TM-align failed for {input_path} against {ref_path}")
        return None
    return res.tm_norm_chain1

def analyze_tm_trajectories(genetic_directories, theoretical_directories, reference_pdb, smoothing_sigma=0.5):
    amino_acids = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']
    for amino_acid in amino_acids:
        plt.figure()
        plt.title(f'TMscore Trajectory for Amino Acid {amino_acid}')
        plt.xlabel('k-steps')
        plt.ylabel('TMscore')
        plt.xlim(0, 195)  # Set x-axis limit
        plt.ylim(0, 1)    # Set y-axis limit based on TMscore range

        all_tmscores_genetic = []
        all_tmscores_theoretical = []

        for parent_directory in genetic_directories:
            pdb_files = [f for f in os.listdir(parent_directory) if f.startswith(f'{amino_acid}') and f.endswith('.pdb')]
            pdb_files.sort(key=lambda x: int(re.search(r'\d+', x).group()))

            tmscores = []
            for pdb_file in pdb_files:
                try:
                    tm = tm_score(os.path.join(parent_directory, pdb_file), reference_pdb)
                    if tm is not None:
                        tmscores.append(tm)
                except Exception as e:
                    print(f"Error processing {pdb_file}: {e}")
            all_tmscores_genetic.append(tmscores)

        for parent_directory in theoretical_directories:
            pdb_files = [f for f in os.listdir(parent_directory) if f.startswith(f'{amino_acid}') and f.endswith('.pdb')]
            pdb_files.sort(key=lambda x: int(re.search(r'\d+', x).group()))

            tmscores = []
            for pdb_file in pdb_files:
                try:
                    tm = tm_score(os.path.join(parent_directory, pdb_file), reference_pdb)
                    if tm is not None:
                        tmscores.append(tm)
                except Exception as e:
                    print(f"Error processing {pdb_file}: {e}")
            all_tmscores_theoretical.append(tmscores)

        min_length_genetic = min(len(tmscores) for tmscores in all_tmscores_genetic if tmscores)
        min_length_theoretical = min(len(tmscores) for tmscores in all_tmscores_theoretical if tmscores)

        min_length = min(min_length_genetic, min_length_theoretical)

        trimmed_tmscores_genetic = [tmscores[:min_length] for tmscores in all_tmscores_genetic if tmscores]
        trimmed_tmscores_theoretical = [tmscores[:min_length] for tmscores in all_tmscores_theoretical if tmscores]

        all_tmscores_genetic = np.array(trimmed_tmscores_genetic)
        all_tmscores_theoretical = np.array(trimmed_tmscores_theoretical)

        mean_tmscores_genetic = np.mean(all_tmscores_genetic, axis=0)
        std_tmscores_genetic = np.std(all_tmscores_genetic, axis=0)

        mean_tmscores_theoretical = np.mean(all_tmscores_theoretical, axis=0)
        std_tmscores_theoretical = np.std(all_tmscores_theoretical, axis=0)

        # Apply Gaussian smoothing
        smoothed_mean_genetic = gaussian_filter1d(mean_tmscores_genetic, sigma=smoothing_sigma)
        smoothed_std_genetic = gaussian_filter1d(std_tmscores_genetic, sigma=smoothing_sigma)

        smoothed_mean_theoretical = gaussian_filter1d(mean_tmscores_theoretical, sigma=smoothing_sigma)
        smoothed_std_theoretical = gaussian_filter1d(std_tmscores_theoretical, sigma=smoothing_sigma)

        x_range = range(len(smoothed_mean_genetic))
        plt.plot(x_range, smoothed_mean_genetic, label='TMscore genetic sequences', linewidth=1)
        plt.fill_between(x_range, smoothed_mean_genetic - smoothed_std_genetic, smoothed_mean_genetic + smoothed_std_genetic, alpha=0.3)

        x_range_theoretical = range(len(smoothed_mean_theoretical))
        plt.plot(x_range_theoretical, smoothed_mean_theoretical, label='TMscore theoretical sequences', linewidth=1, linestyle='dashed')
        plt.fill_between(x_range_theoretical, smoothed_mean_theoretical - smoothed_std_theoretical, smoothed_mean_theoretical + smoothed_std_theoretical, alpha=0.3)

        plt.legend()

        plot_dir = 'variant_analysis/PLOTS/combined_theory_genetic/TMscore/'
        if not os.path.exists(plot_dir):
            os.makedirs(plot_dir)

        plot_title = f'TMscore_Trajectory_{amino_acid}'
        plot_filename = os.path.join(plot_dir, f'{plot_title}.png')
        plt.savefig(plot_filename)
        plt.show()

genetic_directories = ['variant_analysis/pdb_metrics/']
theoretical_directories = ['colabfold/metrics/pdb_1/', 'colabfold/metrics/pdb_2/']
reference_pdb = 'ACE2(1).pdb'
analyze_tm_trajectories(genetic_directories, theoretical_directories, reference_pdb)



In [ ]:
def load_structures(input_path, ref_path):
    p = PDBParser(QUIET=True)
    struct_path = Path(input_path)
    ref_path = Path(ref_path)
    parent_dir = struct_path.parent
    key_name = parent_dir.name
    fixed_struct = p.get_structure('fixed', ref_path)
    moving_struct = p.get_structure('moving', struct_path)
    return fixed_struct, moving_struct, key_name

def tm_score(input_path, ref_path):
    ref_struct, query_struct, key_name = load_structures(input_path, ref_path)
    chain = next(ref_struct.get_chains())
    ref_coords, ref_seq = get_residue_data(chain)
    chain = next(query_struct.get_chains())
    coords, seq = get_residue_data(chain)
    res = tm_align(ref_coords, coords, ref_seq, seq)
    if res is None or res.tm_norm_chain1 is None:
        print(f"TM-align failed for {input_path} against {ref_path}")
        return None
    return res.tm_norm_chain1

def analyze_tm_trajectories(genetic_alphafold_directories, genetic_colabfold_directories, theoretical_directories, reference_pdb, smoothing_sigma=0.5):
    amino_acids = ['S']
    for amino_acid in amino_acids:
        plt.figure()
        plt.title(f'TMscore Trajectory for Amino Acid {amino_acid}')
        plt.xlabel('k-steps')
        plt.ylabel('TMscore')
        plt.xlim(0, 195)  # Set x-axis limit
        plt.ylim(0, 1)    # Set y-axis limit based on TMscore range

        def process_directories(directories, label, linestyle, max_length=None):
            all_tmscores = []

            for parent_directory in directories:
                pdb_files = [f for f in os.listdir(parent_directory) if f.startswith(f'{amino_acid}') and f.endswith('.pdb')]
                pdb_files.sort(key=lambda x: int(re.search(r'\d+', x).group()))

                tmscores = []
                for pdb_file in pdb_files:
                    try:
                        tm = tm_score(os.path.join(parent_directory, pdb_file), reference_pdb)
                        if tm is not None:
                            tmscores.append(tm)
                    except Exception as e:
                        print(f"Error processing {pdb_file}: {e}")
                all_tmscores.append(tmscores)

            # Trim lists to the minimum length or to max_length if specified
            if max_length is not None:
                min_length = min(len(tmscores) for tmscores in all_tmscores + [[0]*max_length])
                min_length = min(min_length, max_length)
            else:
                min_length = min(len(tmscores) for tmscores in all_tmscores)
            
            trimmed_tmscores = [tmscores[:min_length] for tmscores in all_tmscores]

            all_tmscores = np.array(trimmed_tmscores)

            mean_tmscores = np.mean(all_tmscores, axis=0)
            std_tmscores = np.std(all_tmscores, axis=0)

            # Apply Gaussian smoothing
            smoothed_mean = gaussian_filter1d(mean_tmscores, sigma=smoothing_sigma)
            smoothed_std = gaussian_filter1d(std_tmscores, sigma=smoothing_sigma)

            x_range = range(len(smoothed_mean))
            plt.plot(x_range, smoothed_mean, label=label, linewidth=1, linestyle=linestyle)
            plt.fill_between(x_range, smoothed_mean - smoothed_std, smoothed_mean + smoothed_std, alpha=0.3)

        # Determine the minimum length of ColabFold sequences
        colabfold_lengths = []
        for parent_directory in genetic_colabfold_directories:
            pdb_files = [f for f in os.listdir(parent_directory) if f.startswith(f'{amino_acid}') and f.endswith('.pdb')]
            colabfold_lengths.append(len(pdb_files))
        min_colabfold_length = min(colabfold_lengths)

        # Process genetic sequences from AlphaFold and ColabFold
        
        process_directories(genetic_colabfold_directories, 'Genetic sequences (ColabFold)', '-', max_length=min_colabfold_length)

        # Process theoretical sequences
        process_directories(theoretical_directories, 'Theoretical sequences', ':', max_length=min_colabfold_length)

        process_directories(genetic_alphafold_directories, 'Genetic sequences (AlphaFold)', '-', max_length=min_colabfold_length)
        
        plt.legend()

        plot_dir = 'variant_analysis/PLOTS/combined_theory_genetic_alphafold/TMscore/'
        if not os.path.exists(plot_dir):
            os.makedirs(plot_dir)

        plot_title = f'TMscore_Trajectory_{amino_acid}'
        plot_filename = os.path.join(plot_dir, f'{plot_title}.png')
        plt.savefig(plot_filename)
        plt.show()

# Example usage:
genetic_alphafold_directories = ['variant_analysis/pdb_metrics_alphafold_onlyPDB/']
genetic_colabfold_directories = ['variant_analysis/pdb_metrics/']
theoretical_directories = ['colabfold/metrics/pdb_1/', 'colabfold/metrics/pdb_2/']
reference_pdb = 'ACE2(1).pdb'

analyze_tm_trajectories(genetic_alphafold_directories, genetic_colabfold_directories, theoretical_directories, reference_pdb)


# SASA along trajectories

In [ ]:
def analyze_sasa_trajectories(genetic_directories, theoretical_directories, smoothing_sigma=0.5):
    amino_acids = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']
    for amino_acid in amino_acids:
        plt.figure()
        plt.title(f'SASA Trajectory for Amino Acid {amino_acid}')
        plt.xlabel('k-steps')
        plt.ylabel('SASA')
        plt.xlim(0, 195)  # Set x-axis limit
        plt.ylim(9000, 37000)  # Set y-axis limit based on expected SASA range

        all_sasa_values_genetic = []
        all_sasa_values_theoretical = []

        for parent_directory in genetic_directories:
            pdb_files = [f for f in os.listdir(parent_directory) if f.startswith(f'{amino_acid}') and f.endswith('.pdb')]
            pdb_files.sort(key=lambda x: int(re.search(r'\d+', x).group()))

            sasa_values = [sasa(os.path.join(parent_directory, pdb_file)) for pdb_file in pdb_files]
            all_sasa_values_genetic.append(sasa_values)

        for parent_directory in theoretical_directories:
            pdb_files = [f for f in os.listdir(parent_directory) if f.startswith(f'{amino_acid}') and f.endswith('.pdb')]
            pdb_files.sort(key=lambda x: int(re.search(r'\d+', x).group()))

            sasa_values = [sasa(os.path.join(parent_directory, pdb_file)) for pdb_file in pdb_files]
            all_sasa_values_theoretical.append(sasa_values)

        min_length_genetic = min(len(sasa_values) for sasa_values in all_sasa_values_genetic)
        min_length_theoretical = min(len(sasa_values) for sasa_values in all_sasa_values_theoretical)

        min_length = min(min_length_genetic, min_length_theoretical)

        trimmed_sasa_values_genetic = [sasa_values[:min_length] for sasa_values in all_sasa_values_genetic]
        trimmed_sasa_values_theoretical = [sasa_values[:min_length] for sasa_values in all_sasa_values_theoretical]

        all_sasa_values_genetic = np.array(trimmed_sasa_values_genetic)
        all_sasa_values_theoretical = np.array(trimmed_sasa_values_theoretical)

        mean_sasa_genetic = np.mean(all_sasa_values_genetic, axis=0)
        std_sasa_genetic = np.std(all_sasa_values_genetic, axis=0)

        mean_sasa_theoretical = np.mean(all_sasa_values_theoretical, axis=0)
        std_sasa_theoretical = np.std(all_sasa_values_theoretical, axis=0)

        # Apply Gaussian smoothing
        smoothed_mean_genetic = gaussian_filter1d(mean_sasa_genetic, sigma=smoothing_sigma)
        smoothed_std_genetic = gaussian_filter1d(std_sasa_genetic, sigma=smoothing_sigma)

        smoothed_mean_theoretical = gaussian_filter1d(mean_sasa_theoretical, sigma=smoothing_sigma)
        smoothed_std_theoretical = gaussian_filter1d(std_sasa_theoretical, sigma=smoothing_sigma)

        x_range = range(len(smoothed_mean_genetic))
        plt.plot(x_range, smoothed_mean_genetic, label='SASA genetic sequences', linewidth=1)
        plt.fill_between(x_range, smoothed_mean_genetic - smoothed_std_genetic, smoothed_mean_genetic + smoothed_std_genetic, alpha=0.3)

        x_range_theoretical = range(len(smoothed_mean_theoretical))
        plt.plot(x_range_theoretical, smoothed_mean_theoretical, label='SASA theoretical sequences', linewidth=1, linestyle='dashed')
        plt.fill_between(x_range_theoretical, smoothed_mean_theoretical - smoothed_std_theoretical, smoothed_mean_theoretical + smoothed_std_theoretical, alpha=0.3)

        plt.legend()

        plot_dir = 'variant_analysis/PLOTS/combined_theory_genetic/SASA/'
        if not os.path.exists(plot_dir):
            os.makedirs(plot_dir)

        plot_title = f'SASA_Trajectory_{amino_acid}'
        plot_filename = os.path.join(plot_dir, f'{plot_title}.png')
        plt.savefig(plot_filename)
        plt.show()

genetic_directories = ['variant_analysis/pdb_metrics/']
theoretical_directories = ['colabfold/metrics/pdb_1/', 'colabfold/metrics/pdb_2/']
reference_pdb = 'ACE2(1).pdb'
analyze_sasa_trajectories(genetic_directories, theoretical_directories)


In [ ]:
def analyze_sasa_trajectories(genetic_alphafold_directories, genetic_colabfold_directories, theoretical_directories, smoothing_sigma=0.5):
    amino_acids = ['A', 'P', 'S']
    for amino_acid in amino_acids:
        plt.figure()
        plt.title(f'SASA Trajectory for Amino Acid {amino_acid}')
        plt.xlabel('k-steps')
        plt.ylabel('SASA')
        plt.xlim(0, 195)  # Set x-axis limit
        plt.ylim(9000, 37000)  # Set y-axis limit based on expected SASA range

        def process_directories(directories, label, linestyle, max_length=None):
            all_sasa_values = []

            for parent_directory in directories:
                pdb_files = [f for f in os.listdir(parent_directory) if f.startswith(f'{amino_acid}') and f.endswith('.pdb')]
                pdb_files.sort(key=lambda x: int(re.search(r'\d+', x).group()))

                sasa_values = [sasa(os.path.join(parent_directory, pdb_file)) for pdb_file in pdb_files]
                all_sasa_values.append(sasa_values)

            # Trim lists to the minimum length or to max_length if specified
            if max_length is not None:
                min_length = min(len(sasa_values) for sasa_values in all_sasa_values + [[0]*max_length])
                min_length = min(min_length, max_length)
            else:
                min_length = min(len(sasa_values) for sasa_values in all_sasa_values)
            
            trimmed_sasa_values = [sasa_values[:min_length] for sasa_values in all_sasa_values]

            all_sasa_values = np.array(trimmed_sasa_values)

            mean_sasa = np.mean(all_sasa_values, axis=0)
            std_sasa = np.std(all_sasa_values, axis=0)

            # Apply Gaussian smoothing
            smoothed_mean = gaussian_filter1d(mean_sasa, sigma=smoothing_sigma)
            smoothed_std = gaussian_filter1d(std_sasa, sigma=smoothing_sigma)

            x_range = range(len(smoothed_mean))
            plt.plot(x_range, smoothed_mean, label=label, linewidth=1, linestyle=linestyle)
            plt.fill_between(x_range, smoothed_mean - smoothed_std, smoothed_mean + smoothed_std, alpha=0.3)

        # Determine the minimum length of ColabFold sequences
        colabfold_lengths = []
        for parent_directory in genetic_colabfold_directories:
            pdb_files = [f for f in os.listdir(parent_directory) if f.startswith(f'{amino_acid}') and f.endswith('.pdb')]
            colabfold_lengths.append(len(pdb_files))
        min_colabfold_length = min(colabfold_lengths)

        # Process genetic sequences from AlphaFold and ColabFold
        
        process_directories(genetic_colabfold_directories, 'SASA genetic sequences (ColabFold)', '-', max_length=min_colabfold_length)

        # Process theoretical sequences
        process_directories(theoretical_directories, 'SASA theoretical sequences', ':', max_length=min_colabfold_length)
        
        process_directories(genetic_alphafold_directories, 'SASA genetic sequences (AlphaFold)', '-', max_length=min_colabfold_length)

        plt.legend()

        plot_dir = 'variant_analysis/PLOTS/combined_theory_genetic_alphafold/SASA/'
        if not os.path.exists(plot_dir):
            os.makedirs(plot_dir)

        plot_title = f'SASA_Trajectory_{amino_acid}'
        plot_filename = os.path.join(plot_dir, f'{plot_title}.png')
        plt.savefig(plot_filename)
        plt.show()

# Example usage:
genetic_alphafold_directories = ['variant_analysis/pdb_metrics_alphafold_onlyPDB/']
genetic_colabfold_directories = ['variant_analysis/pdb_metrics/']
theoretical_directories = ['colabfold/metrics/pdb_1/', 'colabfold/metrics/pdb_2/']

analyze_sasa_trajectories(genetic_alphafold_directories, genetic_colabfold_directories, theoretical_directories)


# Spoke Wheel ColabFold

In [ ]:
def extract_plddt(pdb_file):
    plddts = []
    with open(pdb_file, 'r') as file:
        for line in file:
            if line.startswith('ATOM') and line[13:15].strip() == 'CA':
                pldtt = float(line[60:66].strip())
                plddts.append(pldtt)
    return plddts

def calculate_average_plddt(pdb_file):
    plddts = extract_plddt(pdb_file)
    average_plddt = np.mean(plddts)
    return average_plddt

def analyze_trajectories(parent_directories, smoothing_sigma=0.5):
    amino_acids = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']
    all_mean_plddts = {}

    for amino_acid in amino_acids:
        all_plddts = []

        for parent_directory in parent_directories:
            pdb_files = [f for f in os.listdir(parent_directory) if f.startswith(f'{amino_acid}') and f.endswith('.pdb')]
            pdb_files.sort(key=lambda x: int(re.search(r'\d+', x).group()))

            plddts = [calculate_average_plddt(os.path.join(parent_directory, pdb_file)) for pdb_file in pdb_files]
            all_plddts.append(plddts)

        # Trim all lists to the minimum length
        min_length = min(len(plddts) for plddts in all_plddts)
        if min_length == 0:
            continue  # Skip if no valid data found for this amino acid

        trimmed_plddts = [plddts[:min_length] for plddts in all_plddts]

        all_plddts = np.array(trimmed_plddts)
        mean_plddts = np.mean(all_plddts, axis=0)

        # Apply Gaussian smoothing
        smoothed_mean = gaussian_filter1d(mean_plddts, sigma=smoothing_sigma)
        
        all_mean_plddts[amino_acid] = smoothed_mean
    
    return all_mean_plddts

def get_color(value):
    if value >= 90:
        return 'darkblue'
    elif value >= 70:
        return 'lightblue'
    elif value >= 50:
        return 'yellow'
    else:
        return 'orange'

def plot_radar_plot(all_mean_plddts):
    categories = list(all_mean_plddts.keys())
    num_vars = len(categories)
    angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()
    angles += angles[:1]

    fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(polar=True))
    ax.set_theta_offset(np.pi / 2)
    ax.set_theta_direction(-1)

    plt.xticks(angles[:-1], categories)
    ax.yaxis.set_visible(False)

    for i, (amino_acid, scores) in enumerate(all_mean_plddts.items()):
        angle = angles[i]
        for j in range(len(scores)):
            radius = j + 1  # Start from 1 to leave the center empty
            color = get_color(scores[j])
            ax.plot([angle, angle], [radius - 1, radius], color=color, linewidth=2)
    
    # Create custom legend
    from matplotlib.lines import Line2D
    legend_elements = [
        Line2D([0], [0], color='orange', lw=4, label='pLDDT ∈ [0, 50)'),
        Line2D([0], [0], color='yellow', lw=4, label='pLDDT ∈ [50, 70)'),
        Line2D([0], [0], color='lightblue', lw=4, label='pLDDT ∈ [70, 90)'),
        Line2D([0], [0], color='darkblue', lw=4, label='pLDDT ∈ [90, 100]')
    ]
    ax.legend(handles=legend_elements, loc='upper right', bbox_to_anchor=(1.3, 1.1))

    ax.grid(False)
    plt.title('pLDDT Scores for Amino Acid Trajectories', size=20, pad=30)
    plt.show()

# Directories to analyze
parent_directories = ['colabfold/metrics/pdb_1/', 'colabfold/metrics/pdb_2/']
all_mean_plddts = analyze_trajectories(parent_directories)

# Plot the radar plot
plot_radar_plot(all_mean_plddts)


In [ ]:
def load_structures(input_path, ref_path):
    parser = PDBParser(QUIET=True)
    fixed_struct = parser.get_structure('fixed', ref_path)
    moving_struct = parser.get_structure('moving', input_path)
    return fixed_struct, moving_struct, 'E'

def calculate_rmsd(fixed_atoms, moving_atoms, structure):
    super_imposer = Superimposer()
    super_imposer.set_atoms(fixed_atoms, moving_atoms)
    super_imposer.apply(structure.get_atoms())
    return super_imposer.rms

def rmsd(input_path, ref_path):
    fixed_struct, moving_struct, key_name = load_structures(input_path, ref_path)
    fixed_model = fixed_struct[0]
    for chain in fixed_model:
        if chain.id == 'E':
            all_atoms_fixed = [atom for residue in chain.get_residues() for atom in residue]
            all_atoms_moving = [atom for residue in moving_struct.get_residues() for atom in residue]
            if len(all_atoms_fixed) > len(all_atoms_moving):
                dif = len(all_atoms_fixed) - len(all_atoms_moving)
                all_atoms_fixed = all_atoms_fixed[:len(all_atoms_fixed) - dif]
            else:
                dif = len(all_atoms_moving) - len(all_atoms_fixed)
                all_atoms_moving = all_atoms_moving[:len(all_atoms_moving) - dif]
            assert len(all_atoms_fixed) == len(all_atoms_moving)
            rmsd_value = calculate_rmsd(all_atoms_fixed, all_atoms_moving, chain)
            return rmsd_value
    return None

def analyze_trajectories(parent_directories, reference_pdb, smoothing_sigma=0.5):
    amino_acids = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']
    all_mean_rmsd = {}

    for amino_acid in amino_acids:
        all_rmsd_values = []

        for parent_directory in parent_directories:
            pdb_files = [f for f in os.listdir(parent_directory) if f.startswith(f'{amino_acid}') and f.endswith('.pdb')]
            pdb_files.sort(key=lambda x: int(re.search(r'\d+', x).group()))

            rmsd_values = [rmsd(os.path.join(parent_directory, pdb_file), reference_pdb) for pdb_file in pdb_files]
            all_rmsd_values.append(rmsd_values)

        # Trim all lists to the minimum length
        min_length = min(len(rmsd_values) for rmsd_values in all_rmsd_values)
        if min_length == 0:
            continue  # Skip if no valid data found for this amino acid

        trimmed_rmsd_values = [rmsd_values[:min_length] for rmsd_values in all_rmsd_values]

        all_rmsd_values = np.array(trimmed_rmsd_values)
        mean_rmsd = np.mean(all_rmsd_values, axis=0)

        # Apply Gaussian smoothing
        smoothed_mean = gaussian_filter1d(mean_rmsd, sigma=smoothing_sigma)
        
        all_mean_rmsd[amino_acid] = smoothed_mean
    
    return all_mean_rmsd

def get_color(value):
    if value >= 80:
        return 'darkred'
    elif value >= 60:
        return 'red'
    elif value >= 40:
        return 'orange'
    elif value >= 20:
        return 'yellow'
    else:
        return 'green'

def plot_radar_plot(all_mean_rmsd):
    categories = list(all_mean_rmsd.keys())
    num_vars = len(categories)
    angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()
    angles += angles[:1]

    fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(polar=True))
    ax.set_theta_offset(np.pi / 2)
    ax.set_theta_direction(-1)

    plt.xticks(angles[:-1], categories)
    ax.yaxis.set_visible(False)

    for i, (amino_acid, scores) in enumerate(all_mean_rmsd.items()):
        angle = angles[i]
        for j in range(len(scores)):
            radius = j + 1  # Start from 1 to leave the center empty
            color = get_color(scores[j])
            ax.plot([angle, angle], [radius - 1, radius], color=color, linewidth=2)
    
    # Create custom legend
    from matplotlib.lines import Line2D
    legend_elements = [
        Line2D([0], [0], color='green', lw=4, label='RMSD ∈ [0-20)'),
        Line2D([0], [0], color='yellow', lw=4, label='RMSD ∈ [20-40)'),
        Line2D([0], [0], color='orange', lw=4, label='RMSD ∈ [40-60)'),
        Line2D([0], [0], color='red', lw=4, label='RMSD ∈ [60-80)'),
        Line2D([0], [0], color='darkred', lw=4, label='RMSD ∈ [80-100]')
    ]
    ax.legend(handles=legend_elements, loc='upper right', bbox_to_anchor=(1.3, 1.1))

    ax.grid(False)
    plt.title('RMSD Scores for Amino Acid Trajectories', size=20, pad=30)
    plt.show()

# Directories to analyze
parent_directories = ['colabfold/metrics/pdb_1/', 'colabfold/metrics/pdb_2/']
reference_pdb = 'ACE2(1).pdb'
all_mean_rmsd = analyze_trajectories(parent_directories, reference_pdb)

# Plot the radar plot
plot_radar_plot(all_mean_rmsd)


In [ ]:
def analyze_trajectories(parent_directories, reference_pdb, smoothing_sigma=0.5):
    amino_acids = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']
    all_mean_tm_scores = {}

    for amino_acid in amino_acids:
        all_tm_scores = []

        for parent_directory in parent_directories:
            pdb_files = [f for f in os.listdir(parent_directory) if f.startswith(f'{amino_acid}') and f.endswith('.pdb')]
            pdb_files.sort(key=lambda x: int(re.search(r'\d+', x).group()))

            tm_scores = [tm_score(os.path.join(parent_directory, pdb_file), reference_pdb) for pdb_file in pdb_files]
            all_tm_scores.append(tm_scores)

        # Trim all lists to the minimum length
        min_length = min(len(tm_scores) for tm_scores in all_tm_scores)
        if min_length == 0:
            continue  # Skip if no valid data found for this amino acid

        trimmed_tm_scores = [tm_scores[:min_length] for tm_scores in all_tm_scores]

        all_tm_scores = np.array(trimmed_tm_scores)
        mean_tm_scores = np.mean(all_tm_scores, axis=0)

        # Apply Gaussian smoothing
        smoothed_mean = gaussian_filter1d(mean_tm_scores, sigma=smoothing_sigma)
        
        all_mean_tm_scores[amino_acid] = smoothed_mean
    
    return all_mean_tm_scores

def get_color(value):
    if value >= 0.8:
        return 'darkblue'
    elif value >= 0.6:
        return 'blue'
    elif value >= 0.4:
        return 'lightblue'
    elif value >= 0.2:
        return 'lightgreen'
    else:
        return 'green'

def plot_radar_plot(all_mean_tm_scores):
    categories = list(all_mean_tm_scores.keys())
    num_vars = len(categories)
    angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()
    angles += angles[:1]

    fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(polar=True))
    ax.set_theta_offset(np.pi / 2)
    ax.set_theta_direction(-1)

    plt.xticks(angles[:-1], categories)
    ax.yaxis.set_visible(False)

    for i, (amino_acid, scores) in enumerate(all_mean_tm_scores.items()):
        angle = angles[i]
        for j in range(len(scores)):
            radius = j + 1  # Start from 1 to leave the center empty
            color = get_color(scores[j])
            ax.plot([angle, angle], [radius - 1, radius], color=color, linewidth=2)
    
    # Create custom legend
    from matplotlib.lines import Line2D
    legend_elements = [
        Line2D([0], [0], color='green', lw=4, label='TM-score ∈ [0-0.2)'),
        Line2D([0], [0], color='lightgreen', lw=4, label='TM-score ∈ [0.2-0.4)'),
        Line2D([0], [0], color='lightblue', lw=4, label='TM-score ∈ [0.4-0.6)'),
        Line2D([0], [0], color='blue', lw=4, label='TM-score ∈ [0.6-0.8)'),
        Line2D([0], [0], color='darkblue', lw=4, label='TM-score ∈ [0.8-1]')
    ]
    ax.legend(handles=legend_elements, loc='upper right', bbox_to_anchor=(1.3, 1.1))

    ax.grid(False)
    plt.title('TM Scores for Amino Acid Trajectories', size=20, pad=30)
    plt.show()

# Directories to analyze
parent_directories = ['colabfold/metrics/pdb_1/', 'colabfold/metrics/pdb_2/']
reference_pdb = 'ACE2(1).pdb'
all_mean_tm_scores = analyze_trajectories(parent_directories, reference_pdb)

# Plot the radar plot
plot_radar_plot(all_mean_tm_scores)

In [ ]:
def analyze_sasa_trajectories(parent_directories, smoothing_sigma=0.5):
    amino_acids = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']
    all_mean_sasa = {}

    for amino_acid in amino_acids:
        all_sasa_values = []

        for parent_directory in parent_directories:
            pdb_files = [f for f in os.listdir(parent_directory) if f.startswith(f'{amino_acid}') and f.endswith('.pdb')]
            pdb_files.sort(key=lambda x: int(re.search(r'\d+', x).group()))

            sasa_values = [sasa(os.path.join(parent_directory, pdb_file)) for pdb_file in pdb_files]
            all_sasa_values.append(sasa_values)

        # Trim all lists to the minimum length
        min_length = min(len(sasa_values) for sasa_values in all_sasa_values)
        if min_length == 0:
            continue  # Skip if no valid data found for this amino acid

        trimmed_sasa_values = [sasa_values[:min_length] for sasa_values in all_sasa_values]

        all_sasa_values = np.array(trimmed_sasa_values)
        mean_sasa = np.mean(all_sasa_values, axis=0)

        # Apply Gaussian smoothing
        smoothed_mean = gaussian_filter1d(mean_sasa, sigma=smoothing_sigma)
        
        all_mean_sasa[amino_acid] = smoothed_mean
    
    return all_mean_sasa

def get_color(value):
    if value >= 40000:
        return 'darkred'
    elif value >= 30000:
        return 'red'
    elif value >= 20000:
        return 'orange'
    elif value >= 10000:
        return 'yellow'
    else:
        return 'green'

def plot_radar_plot(all_mean_sasa):
    categories = list(all_mean_sasa.keys())
    num_vars = len(categories)
    angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()
    angles += angles[:1]

    fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(polar=True))
    ax.set_theta_offset(np.pi / 2)
    ax.set_theta_direction(-1)

    plt.xticks(angles[:-1], categories)
    ax.yaxis.set_visible(False)

    for i, (amino_acid, scores) in enumerate(all_mean_sasa.items()):
        angle = angles[i]
        for j in range(len(scores)):
            radius = j + 1  # Start from 1 to leave the center empty
            color = get_color(scores[j])
            ax.plot([angle, angle], [radius - 1, radius], color=color, linewidth=2)
    
    # Create custom legend
    from matplotlib.lines import Line2D
    legend_elements = [
        Line2D([0], [0], color='green', lw=4, label='SASA ∈ [0-10000)'),
        Line2D([0], [0], color='yellow', lw=4, label='SASA ∈ [10000-20000)'),
        Line2D([0], [0], color='orange', lw=4, label='SASA ∈ [20000-30000)'),
        Line2D([0], [0], color='red', lw=4, label='SASA ∈ [30000-40000)'),
        Line2D([0], [0], color='darkred', lw=4, label='SASA ∈ [40000-∞)')
    ]
    ax.legend(handles=legend_elements, loc='upper right', bbox_to_anchor=(1.3, 1.1))

    ax.grid(False)
    plt.title('SASA for Amino Acid Trajectories', size=20, pad=30)
    plt.show()

# Directories to analyze
parent_directories = ['colabfold/metrics/pdb_1/', 'colabfold/metrics/pdb_2/']
all_mean_sasa = analyze_sasa_trajectories(parent_directories)

# Plot the radar plot
plot_radar_plot(all_mean_sasa)

# Spoke Wheel ESMFold

In [ ]:
def extract_plddt(pdb_file):
    plddts = []
    with open(pdb_file, 'r') as file:
        for line in file:
            if line.startswith('ATOM') and line[13:15].strip() == 'CA':
                pldtt = float(line[60:66].strip())
                plddts.append(pldtt)
    return plddts

def calculate_average_plddt(pdb_file):
    plddts = extract_plddt(pdb_file)
    average_plddt = np.mean(plddts)
    return average_plddt

def analyze_trajectories(parent_directories, smoothing_sigma=0.5):
    amino_acids = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']
    all_mean_plddts = {}

    for amino_acid in amino_acids:
        all_plddts = []

        for parent_directory in parent_directories:
            pdb_files = [f for f in os.listdir(parent_directory) if f.startswith(f'{amino_acid}') and f.endswith('.pdb')]
            pdb_files.sort(key=lambda x: int(re.search(r'trajectory_(\d+)\.', x).group(1)))

            plddts = [calculate_average_plddt(os.path.join(parent_directory, pdb_file)) for pdb_file in pdb_files]
            all_plddts.append(plddts)

        # Trim all lists to the minimum length
        min_length = min(len(plddts) for plddts in all_plddts)
        if min_length == 0:
            continue  # Skip if no valid data found for this amino acid

        trimmed_plddts = [plddts[:min_length] for plddts in all_plddts]

        all_plddts = np.array(trimmed_plddts)
        mean_plddts = np.mean(all_plddts, axis=0)

        # Apply Gaussian smoothing
        smoothed_mean = gaussian_filter1d(mean_plddts, sigma=smoothing_sigma)
        
        all_mean_plddts[amino_acid] = smoothed_mean
    
    return all_mean_plddts

def get_color(value):
    if value >= 0.9:
        return 'darkblue'
    elif value >= 0.7:
        return 'lightblue'
    elif value >= 0.5:
        return 'yellow'
    else:
        return 'orange'

def plot_radar_plot(all_mean_plddts):
    categories = list(all_mean_plddts.keys())
    num_vars = len(categories)
    angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()
    angles += angles[:1]

    fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(polar=True))
    ax.set_theta_offset(np.pi / 2)
    ax.set_theta_direction(-1)

    plt.xticks(angles[:-1], categories)
    ax.yaxis.set_visible(False)

    for i, (amino_acid, scores) in enumerate(all_mean_plddts.items()):
        angle = angles[i]
        for j in range(len(scores)):
            radius = j + 1  # Start from 1 to leave the center empty
            color = get_color(scores[j])
            ax.plot([angle, angle], [radius - 1, radius], color=color, linewidth=2)
    
    # Create custom legend
    from matplotlib.lines import Line2D
    legend_elements = [
        Line2D([0], [0], color='orange', lw=4, label='pLDDT ∈ [0, 0.5)'),
        Line2D([0], [0], color='yellow', lw=4, label='pLDDT ∈ [0.5, 0.7)'),
        Line2D([0], [0], color='lightblue', lw=4, label='pLDDT ∈ [0.7, 0.9)'),
        Line2D([0], [0], color='darkblue', lw=4, label='pLDDT ∈ [0.9, 1]')
    ]
    ax.legend(handles=legend_elements, loc='upper right', bbox_to_anchor=(1.3, 1.1))

    ax.grid(False)
    plt.title('pLDDT Scores for Amino Acid Trajectories', size=20, pad=30)
    plt.show()

# Directories to analyze
parent_directories = ['ESM/output_esm/trajectories_1/', 'ESM/output_esm/trajectories_2/', 'ESM/output_esm/trajectories_3/']
all_mean_plddts = analyze_trajectories(parent_directories)

# Plot the radar plot
plot_radar_plot(all_mean_plddts)

In [ ]:
def analyze_trajectories(parent_directories, reference_pdb, smoothing_sigma=0.5):
    amino_acids = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']
    all_mean_tm_scores = {}

    for amino_acid in amino_acids:
        all_tm_scores = []

        for parent_directory in parent_directories:
            pdb_files = [f for f in os.listdir(parent_directory) if f.startswith(f'{amino_acid}') and f.endswith('.pdb')]
            pdb_files.sort(key=lambda x: int(re.search(r'trajectory_(\d+)\.', x).group(1)))

            tm_scores = [tm_score(os.path.join(parent_directory, pdb_file), reference_pdb) for pdb_file in pdb_files]
            all_tm_scores.append(tm_scores)

        # Trim all lists to the minimum length
        min_length = min(len(tm_scores) for tm_scores in all_tm_scores)
        if min_length == 0:
            continue  # Skip if no valid data found for this amino acid

        trimmed_tm_scores = [tm_scores[:min_length] for tm_scores in all_tm_scores]

        all_tm_scores = np.array(trimmed_tm_scores)
        mean_tm_scores = np.mean(all_tm_scores, axis=0)

        # Apply Gaussian smoothing
        smoothed_mean = gaussian_filter1d(mean_tm_scores, sigma=smoothing_sigma)
        
        all_mean_tm_scores[amino_acid] = smoothed_mean
    
    return all_mean_tm_scores

def get_color(value):
    if value >= 0.8:
        return 'darkblue'
    elif value >= 0.6:
        return 'blue'
    elif value >= 0.4:
        return 'lightblue'
    elif value >= 0.2:
        return 'lightgreen'
    else:
        return 'green'

def plot_radar_plot(all_mean_tm_scores):
    categories = list(all_mean_tm_scores.keys())
    num_vars = len(categories)
    angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()
    angles += angles[:1]

    fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(polar=True))
    ax.set_theta_offset(np.pi / 2)
    ax.set_theta_direction(-1)

    plt.xticks(angles[:-1], categories)
    ax.yaxis.set_visible(False)

    for i, (amino_acid, scores) in enumerate(all_mean_tm_scores.items()):
        angle = angles[i]
        for j in range(len(scores)):
            radius = j + 1  # Start from 1 to leave the center empty
            color = get_color(scores[j])
            ax.plot([angle, angle], [radius - 1, radius], color=color, linewidth=2)
    
    # Create custom legend
    from matplotlib.lines import Line2D
    legend_elements = [
        Line2D([0], [0], color='green', lw=4, label='TM-score ∈ [0-0.2)'),
        Line2D([0], [0], color='lightgreen', lw=4, label='TM-score ∈ [0.2-0.4)'),
        Line2D([0], [0], color='lightblue', lw=4, label='TM-score ∈ [0.4-0.6)'),
        Line2D([0], [0], color='blue', lw=4, label='TM-score ∈ [0.6-0.8)'),
        Line2D([0], [0], color='darkblue', lw=4, label='TM-score ∈ [0.8-1]')
    ]
    ax.legend(handles=legend_elements, loc='upper right', bbox_to_anchor=(1.3, 1.1))

    ax.grid(False)
    plt.title('TM Scores for Amino Acid Trajectories', size=20, pad=30)
    plt.show()

# Directories to analyze
parent_directories = ['ESM/output_esm/trajectories_1/', 'ESM/output_esm/trajectories_2/', 'ESM/output_esm/trajectories_3/']
reference_pdb = 'ACE2(1).pdb'
all_mean_tm_scores = analyze_trajectories(parent_directories, reference_pdb)

# Plot the radar plot
plot_radar_plot(all_mean_tm_scores)

In [ ]:

def analyze_trajectories(parent_directories, reference_pdb, smoothing_sigma=0.5):
    amino_acids = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']
    all_mean_rmsd = {}

    for amino_acid in amino_acids:
        all_rmsd_values = []

        for parent_directory in parent_directories:
            pdb_files = [f for f in os.listdir(parent_directory) if f.startswith(f'{amino_acid}') and f.endswith('.pdb')]
            pdb_files.sort(key=lambda x: int(re.search(r'trajectory_(\d+)\.', x).group(1)))

            rmsd_values = [rmsd(os.path.join(parent_directory, pdb_file), reference_pdb) for pdb_file in pdb_files]
            all_rmsd_values.append(rmsd_values)

        # Trim all lists to the minimum length
        min_length = min(len(rmsd_values) for rmsd_values in all_rmsd_values)
        if min_length == 0:
            continue  # Skip if no valid data found for this amino acid

        trimmed_rmsd_values = [rmsd_values[:min_length] for rmsd_values in all_rmsd_values]

        all_rmsd_values = np.array(trimmed_rmsd_values)
        mean_rmsd = np.mean(all_rmsd_values, axis=0)

        # Apply Gaussian smoothing
        smoothed_mean = gaussian_filter1d(mean_rmsd, sigma=smoothing_sigma)
        
        all_mean_rmsd[amino_acid] = smoothed_mean
    
    return all_mean_rmsd

def get_color(value):
    if value >= 80:
        return 'darkred'
    elif value >= 60:
        return 'red'
    elif value >= 40:
        return 'orange'
    elif value >= 20:
        return 'yellow'
    else:
        return 'green'

def plot_radar_plot(all_mean_rmsd):
    categories = list(all_mean_rmsd.keys())
    num_vars = len(categories)
    angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()
    angles += angles[:1]

    fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(polar=True))
    ax.set_theta_offset(np.pi / 2)
    ax.set_theta_direction(-1)

    plt.xticks(angles[:-1], categories)
    ax.yaxis.set_visible(False)

    for i, (amino_acid, scores) in enumerate(all_mean_rmsd.items()):
        angle = angles[i]
        for j in range(len(scores)):
            radius = j + 1  # Start from 1 to leave the center empty
            color = get_color(scores[j])
            ax.plot([angle, angle], [radius - 1, radius], color=color, linewidth=2)
    
    # Create custom legend
    from matplotlib.lines import Line2D
    legend_elements = [
        Line2D([0], [0], color='green', lw=4, label='RMSD ∈ [0-20)'),
        Line2D([0], [0], color='yellow', lw=4, label='RMSD ∈ [20-40)'),
        Line2D([0], [0], color='orange', lw=4, label='RMSD ∈ [40-60)'),
        Line2D([0], [0], color='red', lw=4, label='RMSD ∈ [60-80)'),
        Line2D([0], [0], color='darkred', lw=4, label='RMSD ∈ [80-100]')
    ]
    ax.legend(handles=legend_elements, loc='upper right', bbox_to_anchor=(1.3, 1.1))

    ax.grid(False)
    plt.title('RMSD Scores for Amino Acid Trajectories', size=20, pad=30)
    plt.show()

# Directories to analyze
parent_directories = ['ESM/output_esm/trajectories_1/', 'ESM/output_esm/trajectories_2/', 'ESM/output_esm/trajectories_3/']
reference_pdb = 'ACE2(1).pdb'
all_mean_rmsd = analyze_trajectories(parent_directories, reference_pdb)

# Plot the radar plot
plot_radar_plot(all_mean_rmsd)


In [ ]:
def analyze_sasa_trajectories(parent_directories, smoothing_sigma=0.5):
    amino_acids = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']
    all_mean_sasa = {}

    for amino_acid in amino_acids:
        all_sasa_values = []

        for parent_directory in parent_directories:
            pdb_files = [f for f in os.listdir(parent_directory) if f.startswith(f'{amino_acid}') and f.endswith('.pdb')]
            pdb_files.sort(key=lambda x: int(re.search(r'trajectory_(\d+)\.', x).group(1)))

            sasa_values = [sasa(os.path.join(parent_directory, pdb_file)) for pdb_file in pdb_files]
            all_sasa_values.append(sasa_values)

        # Trim all lists to the minimum length
        min_length = min(len(sasa_values) for sasa_values in all_sasa_values)
        if min_length == 0:
            continue  # Skip if no valid data found for this amino acid

        trimmed_sasa_values = [sasa_values[:min_length] for sasa_values in all_sasa_values]

        all_sasa_values = np.array(trimmed_sasa_values)
        mean_sasa = np.mean(all_sasa_values, axis=0)

        # Apply Gaussian smoothing
        smoothed_mean = gaussian_filter1d(mean_sasa, sigma=smoothing_sigma)
        
        all_mean_sasa[amino_acid] = smoothed_mean
    
    return all_mean_sasa

def get_color(value):
    if value >= 40000:
        return 'darkred'
    elif value >= 30000:
        return 'red'
    elif value >= 20000:
        return 'orange'
    elif value >= 10000:
        return 'yellow'
    else:
        return 'green'

def plot_radar_plot(all_mean_sasa):
    categories = list(all_mean_sasa.keys())
    num_vars = len(categories)
    angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()
    angles += angles[:1]

    fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(polar=True))
    ax.set_theta_offset(np.pi / 2)
    ax.set_theta_direction(-1)

    plt.xticks(angles[:-1], categories)
    ax.yaxis.set_visible(False)

    for i, (amino_acid, scores) in enumerate(all_mean_sasa.items()):
        angle = angles[i]
        for j in range(len(scores)):
            radius = j + 1  # Start from 1 to leave the center empty
            color = get_color(scores[j])
            ax.plot([angle, angle], [radius - 1, radius], color=color, linewidth=2)
    
    # Create custom legend
    from matplotlib.lines import Line2D
    legend_elements = [
        Line2D([0], [0], color='green', lw=4, label='SASA ∈ [0-10000)'),
        Line2D([0], [0], color='yellow', lw=4, label='SASA ∈ [10000-20000)'),
        Line2D([0], [0], color='orange', lw=4, label='SASA ∈ [20000-30000)'),
        Line2D([0], [0], color='red', lw=4, label='SASA ∈ [30000-40000)'),
        Line2D([0], [0], color='darkred', lw=4, label='SASA ∈ [40000-∞)')
    ]
    ax.legend(handles=legend_elements, loc='upper right', bbox_to_anchor=(1.3, 1.1))

    ax.grid(False)
    plt.title('SASA for Amino Acid Trajectories', size=20, pad=30)
    plt.show()

# Directories to analyze
parent_directories = ['ESM/output_esm/trajectories_1/', 'ESM/output_esm/trajectories_2/', 'ESM/output_esm/trajectories_3/']
all_mean_sasa = analyze_sasa_trajectories(parent_directories)

# Plot the radar plot
plot_radar_plot(all_mean_sasa)